Set up environment and load in data

In [ ]:
library(tidyverse)
library(brms)
library(here)
library(grid)
library(gridExtra)
library(lemon)
theme_set(theme_bw())
helpers_path = paste0(here(),'/analysis/helpers/')
inputs_path = paste0(here(),'/inputs/')
source(paste0(helpers_path, '01_clean_behavioral_data.R'))
fig_out_path = paste0(here(), '/outputs/fig/')

Add columns that will be used for plots below.

In [ ]:
data_yn_clean = data_yn_clean %>%
  mutate(correct = ifelse(possiblePayoff>reference & yesChosen == 1, 1, ifelse(possiblePayoff < reference & yesChosen == 0, 1, 0))) %>%
  mutate(type_chr = ifelse(type == 1, "HT", "RE"),
         week = ifelse(week == 1, "Week 1", ifelse(week == 2, "Week 2", ifelse(week == 3, "Week 3", NA))),
         week = factor(week, levels = c("Week 1", "Week 2", "Week 3"))) %>%
  mutate(val_diff = possiblePayoff - reference,
         abs_val_diff = abs(possiblePayoff - reference),
         val_diff_bin = round(val_diff/50),
         val_diff_bin_str = paste0(val_diff_bin*50-25,":",val_diff_bin*50+25),
         val_diff_bin_str = factor(val_diff_bin_str, levels = c("-225:-175", "-175:-125", "-125:-75", "-75:-25", "-25:25", "25:75", "75:125", "125:175", "175:225")))

data_bc_clean = data_bc_clean %>%
  mutate(correct = ifelse(possiblePayoffleft>possiblePayoffright & leftChosen == 1, 1, ifelse(possiblePayoffleft<possiblePayoffright & leftChosen == 0, 1, 0))) %>%
  mutate(type_chr = ifelse(typeLeft == 1, "HT", "RE"),
         week = ifelse(week == 1, "Week 1", ifelse(week == 2, "Week 2", ifelse(week == 3, "Week 3", NA))),
         week = factor(week, levels = c("Week 1", "Week 2", "Week 3"))) %>%
  mutate(val_diff = possiblePayoffleft - possiblePayoffright,
         abs_val_diff = abs(possiblePayoffleft - possiblePayoffright),
         val_diff_bin = round(val_diff/50),
         val_diff_bin_str = paste0(val_diff_bin*50-25,":",val_diff_bin*50+25),
         val_diff_bin_str = factor(val_diff_bin_str, levels = c("-275:-225","-225:-175", "-175:-125", "-125:-75", "-75:-25", "-25:25", "25:75", "75:125", "125:175", "175:225", "225:275"))) %>%
  filter(!is.na(val_diff_bin_str))

# Accuracy across days and types

## Modeling tasks separately

In [ ]:
sub_summary = data_yn_clean %>%
  group_by(day, subnum, type_chr) %>%
  summarise(mean_accuracy = mean(correct),
            sem_accuracy = sd(correct)/sqrt(n()),
            .groups = 'keep') %>%
  mutate(task = "Y/N",
         sum_type = "sub") %>%
  rbind(data_bc_clean %>%
  group_by(day, subnum, type_chr) %>%
  summarise(mean_accuracy = mean(correct),
            sem_accuracy = sd(correct)/sqrt(n()),
            .groups = 'keep') %>%
  mutate(task = "BC",
         sum_type = "sub")) %>%
  mutate(task = factor(task, levels = c("Y/N", "BC")))

group_summary = data_yn_clean %>%
  group_by(day, type_chr) %>%
  summarise(mean_accuracy = mean(correct),
            .groups = 'keep') %>%
  mutate(task = "Y/N",
         sum_type = "group", subnum = 600) %>%
  rbind(data_bc_clean %>%
          group_by(day, type_chr) %>%
          summarise(mean_accuracy = mean(correct),
                    .groups = 'keep') %>%
          mutate(task = "BC",
                 sum_type = "group", subnum = 600)) %>%
  mutate(task = factor(task, levels = c("Y/N", "BC")))

In [ ]:
p = ggplot()+
  geom_point(data = group_summary, aes(day, mean_accuracy, color = type_chr))+

  geom_line(data = group_summary, aes(day, mean_accuracy, color = type_chr))+
  
  geom_line(data = sub_summary, aes(day, mean_accuracy, color = type_chr, linetype = as.factor(subnum)), alpha = .3)+

  facet_wrap(~task)+
  labs(x = "Day", y = "Accuracy", color="", linetype = "")+
  theme(panel.grid.minor.x = element_blank(),
        panel.grid.major.x = element_blank(),
        legend.position = "bottom",
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_x_continuous(breaks=seq(1,11,1))+
  scale_y_continuous(breaks=seq(.7,1,.1), limits = c(.7, 1))+
  scale_color_brewer(palette = "Dark2")+
  scale_linetype_manual(values = rep("solid", 6))+
  guides(linetype = "none")+
  annotate("rect", xmin = 2.75, xmax = 3.25, ymin = .7, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 6.75, xmax = 7.25, ymin = .7, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 10.75, xmax = 11.25, ymin = .7, ymax = 1,
           alpha = .25,fill = "gray")

# ggsave(file=paste0(fig_out_path, 'ynbc_groupAndSub_accuracyOverDaysNoErrorbarsNoLtype.jpg'), p, height = 5, width=12, units="in")
p

In [ ]:

sub_summary = data_yn_clean %>%
  group_by(day, subnum, type_chr) %>%
  summarise(mean_accuracy = mean(correct),
            sem_accuracy = sd(correct)/sqrt(n()),
            .groups = 'keep') %>%
  mutate(task = "Y/N",
         sum_type = "sub")

group_summary = data_yn_clean %>%
  group_by(day, type_chr) %>%
  summarise(mean_accuracy = mean(correct),
            .groups = 'keep') %>%
  mutate(task = "Y/N",
         sum_type = "group", subnum = 600)

p = ggplot()+
  geom_point(data = group_summary, aes(day, mean_accuracy, color = type_chr))+

  geom_line(data = group_summary, aes(day, mean_accuracy, color = type_chr))+
  
  geom_line(data = sub_summary, aes(day, mean_accuracy, color = type_chr, linetype = as.factor(subnum)), alpha = .3)+

  labs(x = "Day", y = "Accuracy", color="", linetype = "")+
  theme(panel.grid.minor.x = element_blank(),
        panel.grid.major.x = element_blank(),
        legend.position = "bottom",
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical",
        axis.text = element_text(size=14),
        legend.text = element_text(size=14),
        axis.title = element_text(size = 16))+
  scale_x_continuous(breaks=seq(1,11,1))+
  scale_y_continuous(breaks=seq(.7,1,.1), limits = c(.7, 1))+
  scale_color_brewer(palette = "Dark2", labels = c("Hyper-trained", "Regular Exposure"))+
  scale_linetype_manual(values = rep("solid", 6))+
  guides(linetype = "none")+
  annotate("rect", xmin = 2.75, xmax = 3.25, ymin = .7, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 6.75, xmax = 7.25, ymin = .7, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 10.75, xmax = 11.25, ymin = .7, ymax = 1,
           alpha = .25,fill = "gray")

# ggsave(file=paste0(fig_out_path, 'yn_groupAndSub_accuracyOverDaysNoErrorbarsNoLtype.jpg'), p, height = 5, width=6, units="in")
p

In [ ]:
fn_draws = paste0(inputs_path, 'yn_acc_draws.csv')

if(file.exists(fn_draws)){
  yn_acc_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_yn_clean$subnum)
  normMax = 1
  normMin = 0
  
  yn_acc_draws = tibble()
  
  for(cur_sub in subnums){
  
    cur_dat = data_yn_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type = as.factor(type),
             norm_day = (normMax - normMin) / (max(day) - min(day)) * (day - max(day)) + (normMax))
    
    cur_m <- brm(formula = correct ~  norm_day * type,
                        data= cur_dat,
                        family = bernoulli(link = "logit"),
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    yn_acc_draws = rbind(yn_acc_draws, cur_draws)
  }
  names(yn_acc_draws) = gsub(":", ".", names(yn_acc_draws))
  write.csv(yn_acc_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}


In [ ]:
fn_draws = paste0(inputs_path, 'bc_acc_draws.csv')

if(file.exists(fn_draws)){
  bc_acc_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_bc_clean$subnum)
  normMax = 1
  normMin = 0
  
  bc_acc_draws = tibble()
  
  for(cur_sub in subnums){
  
    cur_dat = data_bc_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type = as.factor(typeLeft),
             norm_day = (normMax - normMin) / (max(day) - min(day)) * (day - max(day)) + (normMax))
    
    cur_m <- brm(formula = correct ~  norm_day * type,
                        data= cur_dat,
                        family = bernoulli(link = "logit"),
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    bc_acc_draws = rbind(bc_acc_draws, cur_draws)
  }
  names(bc_acc_draws) = gsub(":", ".", names(bc_acc_draws))
  write.csv(bc_acc_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}


In [ ]:
yn_par_summary = as_tibble(yn_acc_draws) %>%
  select(b_Intercept, b_norm_day, b_type1, b_norm_day.type1, subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_day" = "Day [0, 1]",
                    "b_type1" = "Type (RE = 0, HT = 1)",
                    "b_norm_day.type1" = "Day x Type"),
         key = factor(key, levels = c("Intercept", "Day [0, 1]", "Type (RE = 0, HT = 1)", "Day x Type"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "Y/N") 

bc_par_summary = as_tibble(bc_acc_draws) %>%
  select(b_Intercept, b_norm_day, b_type1, b_norm_day.type1, subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_day" = "Day [0, 1]",
                    "b_type1" = "Type (RE = 0, HT = 1)",
                    "b_norm_day.type1" = "Day x Type"),
         key = factor(key, levels = c("Intercept", "Day [0, 1]", "Type (RE = 0, HT = 1)", "Day x Type"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "BC") 

plim = max(abs(bc_par_summary$l95), abs(yn_par_summary$l95) ,bc_par_summary$h95, yn_par_summary$h95)

p = yn_par_summary %>%
  rbind(bc_par_summary) %>%
  ggplot(aes(m, subnum, color = task, alpha = task)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_grid(.~key, scales = 'free_x') +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - Correct ~ Day x Type") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_continuous(limits = c(-plim, plim))+
  scale_color_manual(values = c("red", "blue"))+
  scale_alpha_manual(values = c(1, 1))+
  guides(alpha = "none")

# ggsave(file=paste0(fig_out_path, 'ynbc_acc_ParEsts.jpg'), p, height = 3, width = 9, units="in")
p

## Modeling tasks together

In [ ]:
p = ggplot()+
  geom_point(data = group_summary, aes(day, mean_accuracy, color = task))+
  
  geom_line(data = group_summary, aes(day, mean_accuracy, color = task))+
  
  geom_line(data = sub_summary, aes(day, mean_accuracy, color = task, linetype = as.factor(subnum)), alpha = .3)+

  facet_wrap(~type_chr)+
  labs(x = "Day", y = "Accuracy", color="", linetype = "")+
  theme(panel.grid.minor.x = element_blank(),
        panel.grid.major.x = element_blank(),
        legend.position = "bottom",
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_x_continuous(breaks=seq(1,11,1))+
  scale_y_continuous(breaks=seq(.7,1,.1), limits = c(.7, 1))+
  scale_color_manual(values = c("red", "blue"), limits = c("BC", "Y/N"))+
  scale_linetype_manual(values = rep("solid", 6))+
  guides(linetype = "none")+
  annotate("rect", xmin = 2.75, xmax = 3.25, ymin = .7, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 6.75, xmax = 7.25, ymin = .7, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 10.75, xmax = 11.25, ymin = .7, ymax = 1,
           alpha = .25,fill = "gray")

# ggsave(file=paste0(fig_out_path, 'ynbc_groupAndSub_accuracyOverDaysPanelByType.jpg'), p, height = 5, width=12, units="in")
p

In [ ]:
p = ggplot(data = sub_summary)+
  
  geom_point(aes(day, mean_accuracy, color = task)) +

  geom_line(aes(day, mean_accuracy, color = task), alpha = .3)+
  
  geom_errorbar(aes(x = day, color = task, ymin = mean_accuracy - sem_accuracy, ymax = mean_accuracy + sem_accuracy), width=0, alpha = .3)+

  facet_grid(type_chr~subnum)+
  labs(x = "Day", y = "Accuracy", color="")+
  theme(panel.grid.minor.x = element_blank(),
        panel.grid.major.x = element_blank(),
        legend.position = "bottom",
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_x_continuous(breaks=seq(1,11,1))+
  scale_y_continuous(breaks=seq(.7,1,.1), limits = c(.7, 1))+
  scale_color_manual(values = c("red", "blue"), limits = c("BC", "Y/N"))+
  annotate("rect", xmin = 2.75, xmax = 3.25, ymin = .7, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 6.75, xmax = 7.25, ymin = .7, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 10.75, xmax = 11.25, ymin = .7, ymax = 1,
           alpha = .25,fill = "gray")

# ggsave(file=paste0(fig_out_path, 'ynbc_Sub_accuracyOverDaysPanelByType.jpg'), p, height = 5, width=12, units="in")
p

In [ ]:
fn_draws = paste0(inputs_path, 'ynbc_acc_draws.csv')

if(file.exists(fn_draws)){
  ynbc_acc_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_bc_clean$subnum)
  normMax = 1
  normMin = 0
  
  ynbc_acc_draws = tibble()
  
  for(cur_sub in subnums){
  
    cur_dat1 = data_yn_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type = as.factor(type),
             norm_day = (normMax - normMin) / (max(day) - min(day)) * (day - max(day)) + (normMax),
             task = "YN") %>%
      select(correct, norm_day, type, task)
    
    cur_dat2 = data_bc_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type = as.factor(typeLeft),
             norm_day = (normMax - normMin) / (max(day) - min(day)) * (day - max(day)) + (normMax),
             task = "BC") %>%
      select(correct, norm_day, type, task)
    
    cur_dat = rbind(cur_dat1, cur_dat2) %>%
      mutate(task = factor(task, levels = c("YN", "BC")))
    
    cur_m <- brm(formula = correct ~  norm_day * type * task,
                        data= cur_dat,
                        family = bernoulli(link = "logit"),
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    ynbc_acc_draws = rbind(ynbc_acc_draws, cur_draws)
  }
  names(ynbc_acc_draws) = gsub(":", ".", names(ynbc_acc_draws))
  write.csv(ynbc_acc_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}

In [ ]:
names(as_tibble(ynbc_acc_draws))[grepl("b_", names(as_tibble(ynbc_acc_draws)))]

In [ ]:
ynbc_par_summary = as_tibble(ynbc_acc_draws) %>%
  select(b_Intercept, b_norm_day, b_type1, b_taskBC, b_norm_day.type1, b_norm_day.taskBC, b_type1.taskBC, b_norm_day.type1.taskBC, subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_day" = "Day [0, 1]",
                    "b_type1" = "Type (RE = 0, HT = 1)",
                    "b_taskBC" = "Task (YN = 0, BC = 1)",
                    "b_norm_day.type1" = "Day x Type",
                    "b_norm_day.taskBC" = "Day x Task", 
                    "b_type1.taskBC" = "Type x Task", 
                    "b_norm_day.type1.taskBC" = "Day x Type x Task"),
         key = factor(key, levels = c("Intercept", "Day [0, 1]", "Type (RE = 0, HT = 1)", "Task (YN = 0, BC = 1)", "Day x Type", "Day x Task", "Type x Task", "Day x Type x Task"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum)) 

plim = max(abs(ynbc_par_summary$l95), ynbc_par_summary$h95)

p = ynbc_par_summary %>%
  ggplot(aes(m, subnum)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75), color = "purple")+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75), color ="purple")+
  facet_wrap(~key, scales = 'free_x', nrow = 2) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - Correct ~ Day x Type x Task") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_continuous(limits = c(-plim, plim))

# ggsave(file=paste0(fig_out_path, 'ynbc_acc_SingleModelParEsts.jpg'), p, height = 5, width = 9, units="in")
p

## Accuracy difference over days

In [ ]:
sub_summary = data_yn_clean %>%
  group_by(day, subnum, type_chr) %>%
  summarise(mean_accuracy = mean(correct),
            .groups = 'keep') %>%
  spread(type_chr, mean_accuracy) %>%
  mutate(acc_diff = HT - RE,
         task = "Y/N",
         sum_type = "sub") %>%
  rbind(data_bc_clean %>%
          group_by(day, subnum, type_chr) %>%
          summarise(mean_accuracy = mean(correct),
                    .groups = 'keep') %>%
          spread(type_chr, mean_accuracy) %>%
          mutate(acc_diff = HT - RE,
                 task = "BC",
                 sum_type = "sub")) %>%
  mutate(task = factor(task, levels = c("Y/N", "BC")))

group_summary = data_yn_clean %>%
  group_by(day, type_chr) %>%
  summarise(mean_accuracy = mean(correct),
            .groups = 'keep') %>%
  spread(type_chr, mean_accuracy) %>%
  mutate(acc_diff = HT - RE,
         task = "Y/N",
         sum_type = "group", subnum = 600) %>%
  rbind(data_bc_clean %>%
          group_by(day, type_chr) %>%
          summarise(mean_accuracy = mean(correct),
                    .groups = 'keep') %>%
          spread(type_chr, mean_accuracy) %>%
          mutate(acc_diff = HT - RE,
                 task = "BC",
                 sum_type = "group", subnum = 600)) %>%
  mutate(task = factor(task, levels = c("Y/N", "BC")))

In [ ]:
p = ggplot()+
  geom_point(data = group_summary, aes(day, acc_diff), color = "purple")+
  
  geom_line(data = group_summary, aes(day, acc_diff), color = "purple")+
  
  geom_line(data = sub_summary, aes(day, acc_diff, linetype = as.factor(subnum)), alpha = .3, color = "purple")+

  facet_wrap(~task)+
  labs(x = "Day", y = "Accuracy Difference (HT - RE)", linetype = "")+
  theme(panel.grid.minor.x = element_blank(),
        panel.grid.major.x = element_blank(),
        legend.position = "bottom",
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_x_continuous(breaks=seq(1,11,1))+
  scale_y_continuous(breaks=seq(-0.05,.2,0.05), limits= c(-0.07, .20))+
  scale_linetype_manual(values = rep("solid", 6))+
  guides(linetype = "none")+
  annotate("rect", xmin = 2.75, xmax = 3.25, ymin = -0.07, ymax = .20,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 6.75, xmax = 7.25, ymin = -0.07, ymax = .20,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 10.75, xmax = 11.25, ymin = -0.07, ymax = .20,
           alpha = .25,fill = "gray")

# ggsave(file=paste0(fig_out_path, 'ynbc_groupAndSub_accuracyDiffOverDays.jpg'), p, height = 5, width=12, units="in")
p

# Choice psychometrics

## Plots

In [ ]:
sub_summary = data_yn_clean %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(subnum, time_bin, val_diff_bin_str, type_chr) %>%
  summarise(mean_y = mean(yesChosen),
            sem_y = sd(yesChosen)/sqrt(n()),
            .groups="keep") %>%
  mutate(type_chr = ifelse(type_chr == "HT", "Hyper-trained", "Regular Exposure"))

group_summary = data_yn_clean %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(time_bin, val_diff_bin_str, type_chr) %>%
  summarise(mean_y = mean(yesChosen),
            .groups="keep")%>%
  mutate(type_chr = ifelse(type_chr == "HT", "Hyper-trained", "Regular Exposure"))

In [ ]:
p = ggplot() +
  geom_hline(data = group_summary, aes(yintercept =.5), color="gray")+
  geom_vline(data = group_summary, aes(xintercept = 5), color="gray")+
  geom_point(data = group_summary, aes(val_diff_bin_str, mean_y, color = type_chr, alpha = time_bin, size = time_bin))+
  geom_line(data = group_summary, aes(val_diff_bin_str, mean_y, color = type_chr, group = time_bin, alpha = time_bin)) +
  facet_wrap(~type_chr)+
  # labs(x = "Value Stim - Value Reference", y = "p(Yes)", title="Y/N Task", alpha="", size = "")+
    labs(x = "Value Stim - Value Reference", y = "p(Yes)", alpha="", size = "")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5), size = 14),
        axis.text.y = element_text(size = 14),
        strip.text = element_text(size = 14),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical",
        legend.text = element_text(size=14),
        axis.title = element_text(size = 16))+
  scale_color_brewer(palette = "Dark2")+
  scale_x_discrete(labels=c("-225:-175", "", "-125:-75", "", "-25:25", "", "75:125", "", "175:225"))+
  scale_y_continuous(breaks = seq(0,1,.5))+
  guides(linetype = "none", color = "none")+
  scale_alpha_manual(values = c(1, .6, .5, .3, .2))+
  scale_size_manual(values = c(1.5, 2 , 2.5, 3, 3.5))

# ggsave(file=paste0(fig_out_path, 'yn_group_pYesOverValDiffAcrossTimeSDNSymp.jpg'), p, height = 4, width = 9.5, units="in")

p

In [ ]:
p = ggplot() +
  geom_hline(data = sub_summary, aes(yintercept =.5), color="gray")+
  geom_vline(data = sub_summary, aes(xintercept = 5), color="gray")+
  geom_point(data = sub_summary, aes(val_diff_bin_str, mean_y, color = type_chr, alpha = time_bin, size = time_bin))+
  geom_line(data = sub_summary, aes(val_diff_bin_str, mean_y, color = type_chr, group = time_bin, alpha = time_bin)) +
  geom_errorbar(data = sub_summary, aes(x = val_diff_bin_str, ymin = mean_y-sem_y, ymax = mean_y + sem_y, color = type_chr), width = 0, alpha = .6)+
  facet_grid(type_chr~subnum)+
  labs(x = "Value Stim - Value Reference", y = "p(Yes)", title="Y/N Task", alpha="", size = "")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_brewer(palette = "Dark2")+
  scale_x_discrete(labels=c("-225:-175", "", "-125:-75", "", "-25:25", "", "75:125", "", "175:225"))+
  scale_y_continuous(breaks = seq(0,1,.5))+
  guides(linetype = "none", color = "none")+
  scale_alpha_manual(values = c(1, .6, .5, .3, .2))+
  scale_size_manual(values = c(1.5, 2 , 2.5, 3, 3.5))

# ggsave(file=paste0(fig_out_path, 'yn_subj_pYesOverValDiffAcrossTime.jpg'), p, height = 5, width = 12, units="in")
p

In [ ]:
sub_summary = data_bc_clean %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(subnum, time_bin, val_diff_bin_str, type_chr) %>%
  summarise(mean_y = mean(leftChosen),
            sem_y = sd(leftChosen)/sqrt(n()),
            .groups="keep")

group_summary = data_bc_clean %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(time_bin, val_diff_bin_str, type_chr) %>%
  summarise(mean_y = mean(leftChosen),
            .groups="keep")

In [ ]:
p = ggplot() +
  geom_hline(data = group_summary, aes(yintercept =.5), color="gray")+
  geom_vline(data = group_summary, aes(xintercept = 6), color="gray")+
  geom_point(data = group_summary, aes(val_diff_bin_str, mean_y, color = type_chr, alpha = time_bin, size = time_bin))+
  geom_line(data = group_summary, aes(val_diff_bin_str, mean_y, color = type_chr, group = time_bin, alpha = time_bin)) +
  facet_wrap(~type_chr)+
  labs(x = "Value Left - Value Right", y = "p(Left)", title="BC Task",  alpha="", size = "")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5), size = 12),
        axis.text.y = element_text(size = 12),
        strip.text = element_text(size = 12),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_brewer(palette = "Dark2")+
  scale_x_discrete(labels=c("","-225:-175", "", "-125:-75", "", "-25:25", "", "75:125", "", "175:225",""))+
  scale_y_continuous(breaks = seq(0,1,.5))+
  guides(linetype = "none", color = "none")+
  scale_alpha_manual(values = c(1, .6, .5, .3, .2))+
  scale_size_manual(values = c(1.5, 2 , 2.5, 3, 3.5))

# ggsave(file=paste0(fig_out_path, 'bc_group_pYesOverValDiffAcrossTime.jpg'), p, height = 4, width = 8, units="in")

p

In [ ]:
p = ggplot() +
  geom_hline(data = sub_summary, aes(yintercept =.5), color="gray")+
  geom_vline(data = sub_summary, aes(xintercept = 6), color="gray")+
  geom_point(data = sub_summary, aes(val_diff_bin_str, mean_y, color = type_chr, alpha = time_bin, size = time_bin))+
  geom_line(data = sub_summary, aes(val_diff_bin_str, mean_y, color = type_chr, group = time_bin, alpha = time_bin)) +
  geom_errorbar(data = sub_summary, aes(x = val_diff_bin_str, ymin = mean_y-sem_y, ymax = mean_y + sem_y, color = type_chr), width = 0, alpha = .6)+
  facet_grid(type_chr~subnum)+
  labs(x = "Value Left - Value Right", y = "p(Left)", title="BC Task", alpha="", size = "")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_brewer(palette = "Dark2")+
  scale_x_discrete(labels=c("","-225:-175", "", "-125:-75", "", "-25:25", "", "75:125", "", "175:225",""))+
  scale_y_continuous(breaks = seq(0,1,.5))+
  guides(linetype = "none", color = "none")+
  scale_alpha_manual(values = c(1, .6, .5, .3, .2))+
  scale_size_manual(values = c(1.5, 2 , 2.5, 3, 3.5))

# ggsave(file=paste0(fig_out_path, 'bc_subj_pYesOverValDiffAcrossTime.jpg'), p, height = 5, width = 12, units="in")
p

## Models

In [ ]:
fn_draws = paste0(inputs_path, 'yn_logit_norm_draws.csv')

if(file.exists(fn_draws)){
  yn_logit_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_yn_clean$subnum)
  normMax = 1
  normMinDay = 0
  normMinVal = -1
  
  yn_logit_draws = tibble()
  
  for(cur_sub in subnums){
  
    cur_dat = data_yn_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type = as.factor(type), # Coding of type: doesn't matter if it's a factor vs 0/1 integer
             norm_day = (normMax - normMinDay) / (max(day) - min(day)) * (day - max(day)) + (normMax)) %>%
      group_by(day) %>% # Grouping by day to normalize valdiff matters. This makes sure every day ranges [-1,1]
      mutate(norm_valDiff = (normMax - normMinVal) / (max(val_diff) - min(val_diff)) * (val_diff - max(val_diff)) + (normMax)) %>%
      select(yesChosen, norm_valDiff, norm_day, type)
    
    cur_m <- brm(formula = yesChosen ~ norm_valDiff * norm_day * type,
                        data= cur_dat,
                        family = bernoulli(link = "logit"),
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    yn_logit_draws = rbind(yn_logit_draws, cur_draws)
  }
  names(yn_logit_draws) = gsub(":", ".", names(yn_logit_draws))
  write.csv(yn_logit_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}


In [ ]:
fn_draws = paste0(inputs_path, 'bc_logit_norm_draws.csv')

if(file.exists(fn_draws)){
  bc_logit_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_bc_clean$subnum)
  normMax = 1
  normMinDay = 0
  normMinVal = -1
  
  bc_logit_draws = tibble()
  
  for(cur_sub in subnums){
  
    cur_dat = data_bc_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type = as.factor(typeLeft), # Coding of type: doesn't matter if it's a factor vs 0/1 integer
             norm_day = (normMax - normMinDay) / (max(day) - min(day)) * (day - max(day)) + (normMax)) %>%
      group_by(day) %>% # Grouping by day to normalize valdiff matters. This makes sure every day ranges [-1,1]
      mutate(norm_valDiff = (normMax - normMinVal) / (max(val_diff) - min(val_diff)) * (val_diff - max(val_diff)) + (normMax)) %>%
      select(leftChosen, norm_valDiff, norm_day, type)
    
    cur_m <- brm(formula = leftChosen ~ norm_valDiff * norm_day * type,
                        data= cur_dat,
                        family = bernoulli(link = "logit"),
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    bc_logit_draws = rbind(bc_logit_draws, cur_draws)
  }
  names(bc_logit_draws) = gsub(":", ".", names(bc_logit_draws))
  write.csv(bc_logit_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}


In [ ]:
names(as_tibble(yn_logit_draws))[grepl("b_", names(as_tibble(yn_logit_draws)))]

In [ ]:
names(as_tibble(bc_logit_draws))[grepl("b_", names(as_tibble(bc_logit_draws)))]

In [ ]:
yn_par_summary = as_tibble(yn_logit_draws) %>%
  select(b_Intercept, b_norm_valDiff, b_norm_day, b_type1, b_norm_valDiff.norm_day, b_norm_valDiff.type1, b_norm_day.type1, b_norm_valDiff.norm_day.type1, subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_valDiff"  = "Value Diff [-1, 1]",
                    "b_norm_day" = "Day [0, 1]",
                    "b_type1" = "Type (RE = 0, HT = 1)",
                    "b_norm_valDiff.norm_day" = "Value Diff x Day",
                    "b_norm_valDiff.type1" = "Value Diff x Type",
                    "b_norm_day.type1" = "Day x Type",
                    "b_norm_valDiff.norm_day.type1" = "Value Diff x Day x Type"),
         key = factor(key, levels = c("Intercept", "Value Diff [-1, 1]", "Day [0, 1]", "Type (RE = 0, HT = 1)", "Value Diff x Day", "Value Diff x Type", "Day x Type", "Value Diff x Day x Type"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "Y/N") 

bc_par_summary = as_tibble(bc_logit_draws) %>%
  select(b_Intercept, b_norm_valDiff, b_norm_day, b_type1, b_norm_valDiff.norm_day, b_norm_valDiff.type1, b_norm_day.type1, b_norm_valDiff.norm_day.type1, subnum) %>%
  gather(key, value, -subnum) %>%
    mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_valDiff"  = "Value Diff [-1, 1]",
                    "b_norm_day" = "Day [0, 1]",
                    "b_type1" = "Type (RE = 0, HT = 1)",
                    "b_norm_valDiff.norm_day" = "Value Diff x Day",
                    "b_norm_valDiff.type1" = "Value Diff x Type",
                    "b_norm_day.type1" = "Day x Type",
                    "b_norm_valDiff.norm_day.type1" = "Value Diff x Day x Type"),
         key = factor(key, levels = c("Intercept", "Value Diff [-1, 1]", "Day [0, 1]", "Type (RE = 0, HT = 1)", "Value Diff x Day", "Value Diff x Type", "Day x Type", "Value Diff x Day x Type"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "BC") 


In [ ]:
library(lemon)
# plim = max(abs(bc_par_summary$l95), abs(yn_par_summary$l95) ,bc_par_summary$h95, yn_par_summary$h95)

p = yn_par_summary %>%
  rbind(bc_par_summary) %>%
  ggplot(aes(m, subnum, color = task, alpha = task)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_wrap(~key, scales = 'free_x', nrow = 2) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - Yes/Left ~ Value Diff x Day x Type") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  # scale_x_continuous(limits = c(-plim, plim))+
  scale_x_symmetric(mid=0)+
  scale_color_manual(values = c("red", "blue"))+
  scale_alpha_manual(values = c(1, 1))+
  guides(alpha = "none")

# ggsave(file=paste0(fig_out_path, 'ynbc_normLogit_ParEsts.jpg'), p, height = 5, width = 9, units="in")
p

# Value difference distributions

In [ ]:
p = data_yn_clean %>%
  group_by(subnum, type_chr, val_diff_bin_str) %>%
  summarise(n = n(), .groups = "keep") %>%
  ggplot(aes(val_diff_bin_str, n, fill=type_chr))+
  geom_bar(stat="identity", position = "identity", alpha = .5)+
  facet_grid(type_chr~subnum) +
  labs(x = "Value Stim - Value Reference", y = "# of trials", title="Y/N Task")+
  theme(panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_fill_brewer(palette = "Dark2")+
  scale_x_discrete(labels=c("-225:-175", "", "-125:-75", "", "-25:25", "", "75:125", "", "175:225"))+
  scale_y_continuous(breaks = seq(0,1500,500), limits = c(0, 1600))+
  guides(fill = "none")

# ggsave(file=paste0(fig_out_path, 'yn_subj_valDiffDists.jpg'), p, height = 5, width = 12, units="in")
p

In [ ]:
p = data_yn_clean %>%
  filter(subnum == 601) %>%
  group_by(subnum, day, type_chr, val_diff_bin_str) %>%
  summarise(n = n(), .groups = "keep") %>%
  mutate(day = paste0("Day ", day),
         day = factor(day, levels = c(paste0("Day ", seq(1:11))))) %>%
  ggplot(aes(val_diff_bin_str, n, fill=type_chr))+
  geom_bar(stat="identity", position = "identity", alpha = .5)+
  facet_grid(type_chr~ day) +
  labs(x = "Value Stim - Value Reference", y = "# of trials", title="Y/N Task (sub-601)")+
  theme(panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 25, margin = margin(t = 5)))+
  scale_fill_brewer(palette = "Dark2")+
  scale_x_discrete(labels=c("-225:-175", "", "", "", "-25:25", "", "", "", "175:225"))+
  scale_y_continuous(breaks = seq(0,150,50), limits = c(0, 160))+
  guides(fill = "none")

ggsave(file=paste0(fig_out_path, 'yn_sub-601_valDiffDists.jpg'), p, height = 5, width = 16, units="in")
p

# Continuous increase of value difference effect

In [ ]:
sub_summary = data_yn_clean %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(subnum, time_bin, val_diff_bin_str, type_chr) %>%
  summarise(mean_y = mean(yesChosen),
            sem_y = sd(yesChosen)/sqrt(n()),
            .groups="keep")

group_summary = data_yn_clean %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(time_bin, val_diff_bin_str, type_chr) %>%
  summarise(mean_y = mean(yesChosen),
            .groups="keep")

In [ ]:
p = ggplot() +
  geom_hline(data = group_summary, aes(yintercept =.5), color="gray")+
  geom_vline(data = group_summary, aes(xintercept = 5), color="gray")+
  geom_point(data = group_summary, aes(val_diff_bin_str, mean_y, color = type_chr))+
  geom_line(data = group_summary, aes(val_diff_bin_str, mean_y, color = type_chr, group = time_bin)) +
  facet_grid(time_bin~type_chr)+
  labs(x = "Value Stim - Value Reference", y = "p(Yes)", title="Y/N Task", alpha="", size = "")+
  theme(panel.grid = element_blank(),
        legend.position = "none",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5), size = 12),
        axis.text.y = element_text(size = 12),
        strip.text = element_text(size = 12))+
  scale_color_brewer(palette = "Dark2")+
  scale_x_discrete(labels=c("-225:-175", "", "-125:-75", "", "-25:25", "", "75:125", "", "175:225"))+
  scale_y_continuous(breaks = seq(0,1,.5))

# ggsave(file=paste0(fig_out_path, 'yn_group_pYesOverValDiffAcrossTimeTimeBinFacets.jpg'), p, height = 10, width = 8, units="in")

p

In [ ]:
fn_draws = paste0(inputs_path, 'yn_logit_norm_bydays_draws.csv')

if(file.exists(fn_draws)){
  yn_logit_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_yn_clean$subnum)
  days = list("day 1" = c(1), "day 2" = c(2), "day 3" = c(3), "week 2" = c(4, 5, 6, 7, 8), "week 3" = c(9, 10, 11))
  
  normMax = 1
  normMinDay = 0
  normMinVal = -1
  
  yn_logit_draws = tibble()
  
  for(cur_sub in subnums){
    for (j in 1:length(days)){
      cur_days = days[[j]]
     
       cur_dat = data_yn_clean %>% 
        ungroup() %>%
        filter(subnum == cur_sub) %>%
        mutate(type = as.factor(type), # Coding of type: doesn't matter if it's a factor vs 0/1 integer
               norm_day = (normMax - normMinDay) / (max(day) - min(day)) * (day - max(day)) + (normMax)) %>%
        group_by(day) %>% # Grouping by day to normalize valdiff matters. This makes sure every day ranges [-1,1]
        mutate(norm_valDiff = (normMax - normMinVal) / (max(val_diff) - min(val_diff)) * (val_diff - max(val_diff)) + (normMax)) %>%
        filter(day %in% cur_days) %>%
        ungroup() %>%
        select(yesChosen, norm_valDiff, norm_day, type)
      
      cur_m <- brm(formula = yesChosen ~ norm_valDiff * type,
                   data= cur_dat,
                   family = bernoulli(link = "logit"),
                   warmup = 500,
                   iter = 2500,
                   chains = 4,
                   init = "0",
                   cores = 4,
                   seed = 389253)
      
      cur_draws = as_draws_df(cur_m)
      cur_draws$subnum = cur_sub
      cur_draws$day = names(days)[j]
      
      yn_logit_draws = rbind(yn_logit_draws, cur_draws)
    }
  
  }
  names(yn_logit_draws) = gsub(":", ".", names(yn_logit_draws))
  write.csv(yn_logit_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}


In [ ]:
names(as_tibble(yn_logit_draws))[grepl("b_", names(as_tibble(yn_logit_draws)))]

In [ ]:
yn_par_summary = as_tibble(yn_logit_draws) %>%
  select(subnum, day, b_Intercept, b_norm_valDiff, b_type1, b_norm_valDiff.type1) %>%
  mutate(day = factor(day, levels = c("day 1", "day 2", "day 3", "week 2", "week 3"))) %>%
  gather(key, value, -subnum, -day) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_valDiff"  = "Value Diff [-1, 1]",
                    "b_type1" = "Type (RE = 0, HT = 1)",
                    "b_norm_valDiff.type1" = "Value Diff x Type"),
         key = factor(key, levels = c("Intercept", "Value Diff [-1, 1]", "Type (RE = 0, HT = 1)", "Value Diff x Type"))) %>%
  group_by(subnum, day, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(task = "Y/N")

In [ ]:
fn_draws = paste0(inputs_path, 'bc_logit_norm_bydays_draws.csv')

if(file.exists(fn_draws)){
  bc_logit_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_bc_clean$subnum)
  days = list("day 1" = c(1), "day 2" = c(2), "day 3" = c(3), "week 2" = c(4, 5, 6, 7, 8), "week 3" = c(9, 10, 11))
  
  normMax = 1
  normMinDay = 0
  normMinVal = -1
  
  bc_logit_draws = tibble()
  
  for(cur_sub in subnums){
    for (j in 1:length(days)){
      cur_days = days[[j]]
     
       cur_dat = data_bc_clean %>% 
        ungroup() %>%
        filter(subnum == cur_sub) %>%
        mutate(type = as.factor(typeLeft), # Coding of type: doesn't matter if it's a factor vs 0/1 integer
               norm_day = (normMax - normMinDay) / (max(day) - min(day)) * (day - max(day)) + (normMax)) %>%
        group_by(day) %>% # Grouping by day to normalize valdiff matters. This makes sure every day ranges [-1,1]
        mutate(norm_valDiff = (normMax - normMinVal) / (max(val_diff) - min(val_diff)) * (val_diff - max(val_diff)) + (normMax)) %>%
        filter(day %in% cur_days) %>%
        ungroup() %>%
        select(leftChosen, norm_valDiff, norm_day, type)
      
      cur_m <- brm(formula = leftChosen ~ norm_valDiff * type,
                   data= cur_dat,
                   family = bernoulli(link = "logit"),
                   warmup = 500,
                   iter = 2500,
                   chains = 4,
                   init = "0",
                   cores = 4,
                   seed = 389253)
      
      cur_draws = as_draws_df(cur_m)
      cur_draws$subnum = cur_sub
      cur_draws$day = names(days)[j]
      
      bc_logit_draws = rbind(bc_logit_draws, cur_draws)
    }
  
  }
  names(bc_logit_draws) = gsub(":", ".", names(bc_logit_draws))
  write.csv(bc_logit_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}


In [ ]:
bc_par_summary = as_tibble(bc_logit_draws) %>%
  select(subnum, day, b_Intercept, b_norm_valDiff, b_type1, b_norm_valDiff.type1) %>%
  mutate(day = factor(day, levels = c("day 1", "day 2", "day 3", "week 2", "week 3"))) %>%
  gather(key, value, -subnum, -day) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_valDiff"  = "Value Diff [-1, 1]",
                    "b_type1" = "Type (RE = 0, HT = 1)",
                    "b_norm_valDiff.type1" = "Value Diff x Type"),
         key = factor(key, levels = c("Intercept", "Value Diff [-1, 1]", "Type (RE = 0, HT = 1)", "Value Diff x Type"))) %>%
  group_by(subnum, day, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(task = "BC")

In [ ]:
p = yn_par_summary %>%
  rbind(bc_par_summary) %>%
  mutate(task = factor(task, levels = c("Y/N", "BC"))) %>%
  ggplot(aes(day, m, color=task, alpha = task))+
  geom_point(position = position_dodge(width=.75))+
  geom_errorbar(aes(ymin = l95, ymax = h95), width=.2, position = position_dodge(width=.75))+
  facet_grid(key~subnum, scales="free")+
  theme(panel.grid.major.x = element_blank(),
        panel.grid.minor = element_blank(),
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.position = "bottom",
        legend.box.margin=margin(t = -10))+
  labs(x="", y="Parameter Estimate", title="95% HDI for posterior slopes - Yes/Left ~ Value Diff x Type", color = "")+
  scale_color_manual(values = c("red", "blue"), limits = c("BC", "Y/N"))+
  guides(alpha = "none")+
  scale_alpha_manual(values = c(1,1))

# ggsave(file=paste0(fig_out_path, 'ynbc_logit_norm_parests.jpg'), p, height = 9, width = 10.5, units="in")

p

## Pooled model comparing value difference slope across time bins

Useful [set of notes](https://stats.oarc.ucla.edu/r/library/r-library-contrast-coding-systems-for-categorical-variables/) on various contrast codings.

In [ ]:

# days = list("day 1" = c(1), "day 2" = c(2), "day 3" = c(3), "week 2" = c(4, 5, 6, 7, 8), "week 3" = c(9, 10, 11))
data_yn_clean = data_yn_clean %>%
  mutate(time_bin = recode(day, 
                    `1` = "day 1", 
                    `2` = "day 2",
                    `3` = "day 3",
                    `4` = "week 2",
                    `5` = "week 2",
                    `6` = "week 2",
                    `7` = "week 2",
                    `8` = "week 2",
                    `9` = "week 3",
                    `10` = "week 3",
                    `11` = "week 3"),
         time_bin = factor(time_bin))

data_bc_clean = data_bc_clean %>%
  mutate(time_bin = recode(day, 
                    `1` = "day 1", 
                    `2` = "day 2",
                    `3` = "day 3",
                    `4` = "week 2",
                    `5` = "week 2",
                    `6` = "week 2",
                    `7` = "week 2",
                    `8` = "week 2",
                    `9` = "week 3",
                    `10` = "week 3",
                    `11` = "week 3"),
         time_bin = factor(time_bin))

contrasts(data_yn_clean$time_bin)

### Current to all later

Helmert coding: Compare day 1 to later, day 2 to later etc. 

In [ ]:
k = length(unique(data_yn_clean$time_bin)) # number of factor levels

my_helmert = matrix(c(4/5, -1/5, -1/5, -1/5, -1/5,
                      0, 3/4, -1/4, -1/4, -1/4,
                      0, 0, 2/3, -1/3, -1/3,
                      0, 0, 0, 1/2, -1/2), ncol = k-1)
  
my_helmert

In [ ]:
fn_draws = paste0(inputs_path, 'yn_logit_norm_bydays_helmert_draws.csv')

if(file.exists(fn_draws)){
  yn_logit_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_yn_clean$subnum)

  normMax = 1
  normMinDay = 0
  normMinVal = -1

  yn_logit_draws = tibble()

  for(cur_sub in subnums){

    cur_dat = data_yn_clean %>%
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type = as.factor(type), # Coding of type: doesn't matter if it's a factor vs 0/1 integer
             norm_day = (normMax - normMinDay) / (max(day) - min(day)) * (day - max(day)) + (normMax)) %>%
      group_by(day) %>% # Grouping by day to normalize valdiff matters. This makes sure every day ranges [-1,1]
      mutate(norm_valDiff = (normMax - normMinVal) / (max(val_diff) - min(val_diff)) * (val_diff - max(val_diff)) + (normMax)) %>%
      ungroup() %>%
      select(yesChosen, norm_valDiff, time_bin, type)
    
    contrasts(cur_dat$time_bin) = my_helmert
    
    cur_m <- brm(formula = yesChosen ~ norm_valDiff * type * time_bin,
                 data= cur_dat,
                 family = bernoulli(link = "logit"),
                 warmup = 500,
                 iter = 2500,
                 chains = 4,
                 init = "0",
                 cores = 4,
                 seed = 389253)

    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub

    yn_logit_draws = rbind(yn_logit_draws, cur_draws)

  }
  names(yn_logit_draws) = gsub(":", ".", names(yn_logit_draws))
  write.csv(yn_logit_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}


In [ ]:
fn_draws = paste0(inputs_path, 'bc_logit_norm_bydays_helmert_draws.csv')

if(file.exists(fn_draws)){
  bc_logit_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_bc_clean$subnum)
  normMax = 1
  normMinDay = 0
  normMinVal = -1
  
  bc_logit_draws = tibble()
  
  for(cur_sub in subnums){
  
    cur_dat = data_bc_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type = as.factor(typeLeft), # Coding of type: doesn't matter if it's a factor vs 0/1 integer
             norm_day = (normMax - normMinDay) / (max(day) - min(day)) * (day - max(day)) + (normMax)) %>%
      group_by(day) %>% # Grouping by day to normalize valdiff matters. This makes sure every day ranges [-1,1]
      mutate(norm_valDiff = (normMax - normMinVal) / (max(val_diff) - min(val_diff)) * (val_diff - max(val_diff)) + (normMax)) %>%
      select(leftChosen, norm_valDiff, time_bin, type)
    
    contrasts(cur_dat$time_bin) = my_helmert
    
    cur_m <- brm(formula = leftChosen ~ norm_valDiff * type * time_bin,
                        data= cur_dat,
                        family = bernoulli(link = "logit"),
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    bc_logit_draws = rbind(bc_logit_draws, cur_draws)
  }
  names(bc_logit_draws) = gsub(":", ".", names(bc_logit_draws))
  write.csv(bc_logit_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}


In [ ]:
names(as_tibble(yn_logit_draws))[grepl("b_", names(as_tibble(yn_logit_draws)))]

In [ ]:
names(as_tibble(bc_logit_draws))[grepl("b_", names(as_tibble(bc_logit_draws)))]

In [ ]:
names(as_tibble(yn_logit_draws))

In [ ]:
yn_par_summary = as_tibble(yn_logit_draws) %>%
  select(-lprior, -lp__, -.chain, -.iteration, -.draw) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_valDiff"  = "Value Diff [-1, 1]",
                    "b_type1" = "Type (RE = 0, HT = 1)",
                    "b_time_bin1" = "Day 1 vs Later",
                    "b_time_bin2" = "Day 2 vs Later",
                    "b_time_bin3" = "Day 3 vs Later",
                    "b_time_bin4" = "Week 2 vs Later",
                    "b_norm_valDiff.type1" = "Value Diff x Type",
                    "b_norm_valDiff.time_bin1" = "Value Diff x Day 1 vs Later",
                    "b_norm_valDiff.time_bin2" = "Value Diff x Day 2 vs Later",
                    "b_norm_valDiff.time_bin3" = "Value Diff x Day 3 vs Later",
                    "b_norm_valDiff.time_bin4" = "Value Diff x Week 2 vs Later",
                    "b_type1.time_bin1" = "Type x Day 1 vs Later",
                    "b_type1.time_bin2" = "Type x Day 2 vs Later",
                    "b_type1.time_bin3" = "Type x Day 3 vs Later",
                    "b_type1.time_bin4" = "Type x Week 2 vs Later",
                    "b_norm_valDiff.type1.time_bin1" = "Value Diff x Type x Day 1 vs Later",
                    "b_norm_valDiff.type1.time_bin2" = "Value Diff x Type x Day 2 vs Later",
                    "b_norm_valDiff.type1.time_bin3" = "Value Diff x Type x Day 3 vs Later",
                    "b_norm_valDiff.type1.time_bin4" = "Value Diff x Type x Week 2 vs Later",
                    ),
         key = factor(key, levels = c("Intercept", "Value Diff [-1, 1]", "Type (RE = 0, HT = 1)", "Day 1 vs Later", "Day 2 vs Later", "Day 3 vs Later", "Week 2 vs Later", "Value Diff x Type", "Value Diff x Day 1 vs Later", "Value Diff x Day 2 vs Later", "Value Diff x Day 3 vs Later", "Value Diff x Week 2 vs Later", "Type x Day 1 vs Later", "Type x Day 2 vs Later", "Type x Day 3 vs Later", "Type x Week 2 vs Later", "Value Diff x Type x Day 1 vs Later", "Value Diff x Type x Day 2 vs Later", "Value Diff x Type x Day 3 vs Later", "Value Diff x Type x Week 2 vs Later"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "Y/N") 

bc_par_summary = as_tibble(bc_logit_draws) %>%
  select(-lprior, -lp__, -.chain, -.iteration, -.draw) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_valDiff"  = "Value Diff [-1, 1]",
                    "b_type1" = "Type (RE = 0, HT = 1)",
                    "b_time_bin1" = "Day 1 vs Later",
                    "b_time_bin2" = "Day 2 vs Later",
                    "b_time_bin3" = "Day 3 vs Later",
                    "b_time_bin4" = "Week 2 vs Later",
                    "b_norm_valDiff.type1" = "Value Diff x Type",
                    "b_norm_valDiff.time_bin1" = "Value Diff x Day 1 vs Later",
                    "b_norm_valDiff.time_bin2" = "Value Diff x Day 2 vs Later",
                    "b_norm_valDiff.time_bin3" = "Value Diff x Day 3 vs Later",
                    "b_norm_valDiff.time_bin4" = "Value Diff x Week 2 vs Later",
                    "b_type1.time_bin1" = "Type x Day 1 vs Later",
                    "b_type1.time_bin2" = "Type x Day 2 vs Later",
                    "b_type1.time_bin3" = "Type x Day 3 vs Later",
                    "b_type1.time_bin4" = "Type x Week 2 vs Later",
                    "b_norm_valDiff.type1.time_bin1" = "Value Diff x Type x Day 1 vs Later",
                    "b_norm_valDiff.type1.time_bin2" = "Value Diff x Type x Day 2 vs Later",
                    "b_norm_valDiff.type1.time_bin3" = "Value Diff x Type x Day 3 vs Later",
                    "b_norm_valDiff.type1.time_bin4" = "Value Diff x Type x Week 2 vs Later",
                    ),
         key = factor(key, levels = c("Intercept", "Value Diff [-1, 1]", "Type (RE = 0, HT = 1)", "Day 1 vs Later", "Day 2 vs Later", "Day 3 vs Later", "Week 2 vs Later", "Value Diff x Type", "Value Diff x Day 1 vs Later", "Value Diff x Day 2 vs Later", "Value Diff x Day 3 vs Later", "Value Diff x Week 2 vs Later", "Type x Day 1 vs Later", "Type x Day 2 vs Later", "Type x Day 3 vs Later", "Type x Week 2 vs Later", "Value Diff x Type x Day 1 vs Later", "Value Diff x Type x Day 2 vs Later", "Value Diff x Type x Day 3 vs Later", "Value Diff x Type x Week 2 vs Later"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "BC") 


In [ ]:
p = yn_par_summary %>%
  rbind(bc_par_summary) %>%
  ggplot(aes(m, subnum, color = task, alpha = task)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_wrap(~key, scales = 'free_x', nrow = 2, labeller = labeller(key = label_wrap_gen(10))) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - Yes/Left ~ Value Diff x Time Bin x Type") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_symmetric(mid=0)+
  scale_color_manual(values = c("red", "blue"))+
  scale_alpha_manual(values = c(1, 1))+
  guides(alpha = "none")

# ggsave(file=paste0(fig_out_path, 'ynbc_normLogit_helmert_ParEsts.jpg'), p, height = 6, width = 12, units="in")
p

### Successive levels

Forward difference coding: Compare day 1 to day 2, day 2 to day 3 etc


In [ ]:
my_fwd_diff = matrix(c(4/5, -1/5, -1/5, -1/5, -1/5,
                       3/5, 3/5, -2/5, -2/5, -2/5,
                       2/5, 2/5, 2/5, -3/5, -3/5,
                       1/5, 1/5, 1/5, 1/5, -4/5), ncol = k-1)

my_fwd_diff

In [ ]:
fn_draws = paste0(inputs_path, 'yn_logit_norm_bydays_fwd_diff_draws.csv')

if(file.exists(fn_draws)){
  yn_logit_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_yn_clean$subnum)

  normMax = 1
  normMinDay = 0
  normMinVal = -1

  yn_logit_draws = tibble()

  for(cur_sub in subnums){

    cur_dat = data_yn_clean %>%
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type = as.factor(type), # Coding of type: doesn't matter if it's a factor vs 0/1 integer
             norm_day = (normMax - normMinDay) / (max(day) - min(day)) * (day - max(day)) + (normMax)) %>%
      group_by(day) %>% # Grouping by day to normalize valdiff matters. This makes sure every day ranges [-1,1]
      mutate(norm_valDiff = (normMax - normMinVal) / (max(val_diff) - min(val_diff)) * (val_diff - max(val_diff)) + (normMax)) %>%
      ungroup() %>%
      select(yesChosen, norm_valDiff, time_bin, type)
    
    contrasts(cur_dat$time_bin) = my_fwd_diff
    
    cur_m <- brm(formula = yesChosen ~ norm_valDiff * type * time_bin,
                 data= cur_dat,
                 family = bernoulli(link = "logit"),
                 warmup = 500,
                 iter = 2500,
                 chains = 4,
                 init = "0",
                 cores = 4,
                 seed = 389253)

    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub

    yn_logit_draws = rbind(yn_logit_draws, cur_draws)

  }
  names(yn_logit_draws) = gsub(":", ".", names(yn_logit_draws))
  write.csv(yn_logit_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}


In [ ]:
fn_draws = paste0(inputs_path, 'bc_logit_norm_bydays_fwd_diff_draws.csv')

if(file.exists(fn_draws)){
  bc_logit_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_bc_clean$subnum)
  normMax = 1
  normMinDay = 0
  normMinVal = -1
  
  bc_logit_draws = tibble()
  
  for(cur_sub in subnums){
  
    cur_dat = data_bc_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type = as.factor(typeLeft), # Coding of type: doesn't matter if it's a factor vs 0/1 integer
             norm_day = (normMax - normMinDay) / (max(day) - min(day)) * (day - max(day)) + (normMax)) %>%
      group_by(day) %>% # Grouping by day to normalize valdiff matters. This makes sure every day ranges [-1,1]
      mutate(norm_valDiff = (normMax - normMinVal) / (max(val_diff) - min(val_diff)) * (val_diff - max(val_diff)) + (normMax)) %>%
      select(leftChosen, norm_valDiff, time_bin, type)
    
    contrasts(cur_dat$time_bin) = my_helmert
    
    cur_m <- brm(formula = leftChosen ~ norm_valDiff * type * time_bin,
                        data= cur_dat,
                        family = bernoulli(link = "logit"),
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    bc_logit_draws = rbind(bc_logit_draws, cur_draws)
  }
  names(bc_logit_draws) = gsub(":", ".", names(bc_logit_draws))
  write.csv(bc_logit_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}


In [ ]:
names(as_tibble(yn_logit_draws))[grepl("b_", names(as_tibble(yn_logit_draws)))]

In [ ]:
names(as_tibble(bc_logit_draws))[grepl("b_", names(as_tibble(bc_logit_draws)))]

In [ ]:
names(as_tibble(yn_logit_draws))

In [ ]:
yn_par_summary = as_tibble(yn_logit_draws) %>%
  select(-lprior, -lp__, -.chain, -.iteration, -.draw) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_valDiff"  = "Value Diff [-1, 1]",
                    "b_type1" = "Type (RE = 0, HT = 1)",
                    "b_time_bin1" = "Day 1 vs Day 2",
                    "b_time_bin2" = "Day 2 vs Day 3",
                    "b_time_bin3" = "Day 3 vs Week 2",
                    "b_time_bin4" = "Week 2 vs Week 3",
                    "b_norm_valDiff.type1" = "Value Diff x Type",
                    "b_norm_valDiff.time_bin1" = "Value Diff x Day 1 vs Day 2",
                    "b_norm_valDiff.time_bin2" = "Value Diff x Day 2 vs Day 3",
                    "b_norm_valDiff.time_bin3" = "Value Diff x Day 3 vs Week 2",
                    "b_norm_valDiff.time_bin4" = "Value Diff x Week 2 vs Week 3",
                    "b_type1.time_bin1" = "Type x Day 1 vs Day 2",
                    "b_type1.time_bin2" = "Type x Day 2 vs Day 3",
                    "b_type1.time_bin3" = "Type x Day 3 vs Week 2",
                    "b_type1.time_bin4" = "Type x Week 2 vs Week 3",
                    "b_norm_valDiff.type1.time_bin1" = "Value Diff x Type x Day 1 vs Day 2",
                    "b_norm_valDiff.type1.time_bin2" = "Value Diff x Type x Day 2 vs Day 3",
                    "b_norm_valDiff.type1.time_bin3" = "Value Diff x Type x Day 3 vs Week 2",
                    "b_norm_valDiff.type1.time_bin4" = "Value Diff x Type x Week 2 vs Week 3",
                    ),
         key = factor(key, levels = c("Intercept", "Value Diff [-1, 1]", "Type (RE = 0, HT = 1)", "Day 1 vs Day 2", "Day 2 vs Day 3", "Day 3 vs Week 2", "Week 2 vs Week 3", "Value Diff x Type", "Value Diff x Day 1 vs Day 2", "Value Diff x Day 2 vs Day 3", "Value Diff x Day 3 vs Week 2", "Value Diff x Week 2 vs Week 3", "Type x Day 1 vs Day 2", "Type x Day 2 vs Day 3", "Type x Day 3 vs Week 2", "Type x Week 2 vs Week 3", "Value Diff x Type x Day 1 vs Day 2", "Value Diff x Type x Day 2 vs Day 3", "Value Diff x Type x Day 3 vs Week 2", "Value Diff x Type x Week 2 vs Week 3"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "Y/N") 

bc_par_summary = as_tibble(bc_logit_draws) %>%
  select(-lprior, -lp__, -.chain, -.iteration, -.draw) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_valDiff"  = "Value Diff [-1, 1]",
                    "b_type1" = "Type (RE = 0, HT = 1)",
                    "b_time_bin1" = "Day 1 vs Day 2",
                    "b_time_bin2" = "Day 2 vs Day 3",
                    "b_time_bin3" = "Day 3 vs Week 2",
                    "b_time_bin4" = "Week 2 vs Week 3",
                    "b_norm_valDiff.type1" = "Value Diff x Type",
                    "b_norm_valDiff.time_bin1" = "Value Diff x Day 1 vs Day 2",
                    "b_norm_valDiff.time_bin2" = "Value Diff x Day 2 vs Day 3",
                    "b_norm_valDiff.time_bin3" = "Value Diff x Day 3 vs Week 2",
                    "b_norm_valDiff.time_bin4" = "Value Diff x Week 2 vs Week 3",
                    "b_type1.time_bin1" = "Type x Day 1 vs Day 2",
                    "b_type1.time_bin2" = "Type x Day 2 vs Day 3",
                    "b_type1.time_bin3" = "Type x Day 3 vs Week 2",
                    "b_type1.time_bin4" = "Type x Week 2 vs Week 3",
                    "b_norm_valDiff.type1.time_bin1" = "Value Diff x Type x Day 1 vs Day 2",
                    "b_norm_valDiff.type1.time_bin2" = "Value Diff x Type x Day 2 vs Day 3",
                    "b_norm_valDiff.type1.time_bin3" = "Value Diff x Type x Day 3 vs Week 2",
                    "b_norm_valDiff.type1.time_bin4" = "Value Diff x Type x Week 2 vs Week 3",
                    ),
         key = factor(key, levels = c("Intercept", "Value Diff [-1, 1]", "Type (RE = 0, HT = 1)", "Day 1 vs Day 2", "Day 2 vs Day 3", "Day 3 vs Week 2", "Week 2 vs Week 3", "Value Diff x Type", "Value Diff x Day 1 vs Day 2", "Value Diff x Day 2 vs Day 3", "Value Diff x Day 3 vs Week 2", "Value Diff x Week 2 vs Week 3", "Type x Day 1 vs Day 2", "Type x Day 2 vs Day 3", "Type x Day 3 vs Week 2", "Type x Week 2 vs Week 3", "Value Diff x Type x Day 1 vs Day 2", "Value Diff x Type x Day 2 vs Day 3", "Value Diff x Type x Day 3 vs Week 2", "Value Diff x Type x Week 2 vs Week 3"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "BC") 


In [ ]:
p = yn_par_summary %>%
  rbind(bc_par_summary) %>%
  ggplot(aes(m, subnum, color = task, alpha = task)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_wrap(~key, scales = 'free_x', nrow = 2, labeller = labeller(key = label_wrap_gen(10))) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - Yes/Left ~ Value Diff x Time Bin x Type") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_symmetric(mid=0)+
  scale_color_manual(values = c("red", "blue"))+
  scale_alpha_manual(values = c(1, 1))+
  guides(alpha = "none")

# ggsave(file=paste0(fig_out_path, 'ynbc_normLogit_fwd_diff_ParEsts.jpg'), p, height = 6, width = 12, units="in")
p

# RT

## Modeling tasks separately

In [ ]:
sub_summary = data_yn_clean %>%
  group_by(day, subnum, type_chr) %>%
  summarise(mean_y = mean(log(rt)),
            sem_y = sd(log(rt))/sqrt(n()),
            .groups = 'keep') %>%
  mutate(task = "Y/N",
         sum_type_chr = "sub") %>%
  rbind(data_bc_clean %>%
  group_by(day, subnum, type_chr) %>%
  summarise(mean_y = mean(log(rt)),
            sem_y = sd(log(rt))/sqrt(n()),
            .groups = 'keep') %>%
  mutate(task = "BC",
         sum_type_chr = "sub")) %>%
  mutate(task = factor(task, levels = c("Y/N", "BC")))

group_summary = data_yn_clean %>%
  group_by(day, type_chr) %>%
  summarise(mean_y = mean(log(rt)),
            .groups = 'keep') %>%
  mutate(task = "Y/N",
         sum_type_chr = "group", subnum = 600) %>%
  rbind(data_bc_clean %>%
          group_by(day, type_chr) %>%
          summarise(mean_y = mean(log(rt)),
                    .groups = 'keep') %>%
          mutate(task = "BC",
                 sum_type_chr = "group", subnum = 600)) %>%
  mutate(task = factor(task, levels = c("Y/N", "BC")))

In [ ]:
p = ggplot()+
  geom_point(data = group_summary, aes(day, mean_y, color = type_chr))+

  geom_line(data = group_summary, aes(day, mean_y, color = type_chr))+
  
  geom_line(data = sub_summary, aes(day, mean_y, color = type_chr, linetype = as.factor(subnum)), alpha = .3)+
  
  facet_wrap(~task)+
  guides(linetype_chr = "none")+
  labs(x = "Day", y = "Mean Log RT", color="", shape = "")+
  theme(panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank(),
        panel.grid.minor.y = element_blank(),
        legend.position = "bottom",
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_x_continuous(breaks=seq(1,11,1))+
  scale_color_brewer(palette = "Dark2") +
  scale_linetype_manual(values = rep("solid", 6))+
  guides(linetype = "none")+
  annotate("rect", xmin = 2.75, xmax = 3.25, ymin = -.7, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 6.75, xmax = 7.25, ymin = -.7, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 10.75, xmax = 11.25, ymin = -.7, ymax = 1,
           alpha = .25,fill = "gray")

# ggsave(file=paste0(fig_out_path, 'ynbc_groupAndSub_meanLogRTOverDays.jpg'), p, height = 5, width=12, units="in")
p

In [ ]:
sub_summary = data_yn_clean %>%
  group_by(day, subnum, type_chr) %>%
  summarise(mean_y = mean(log(rt)),
            sem_y = sd(log(rt))/sqrt(n()),
            .groups = 'keep') %>%
  mutate(task = "Y/N",
         sum_type_chr = "sub")

group_summary = data_yn_clean %>%
  group_by(day, type_chr) %>%
  summarise(mean_y = mean(log(rt)),
            .groups = 'keep') %>%
  mutate(task = "Y/N",
         sum_type_chr = "group", subnum = 600)

In [ ]:
p = ggplot()+
  geom_point(data = group_summary, aes(day, mean_y, color = type_chr))+

  geom_line(data = group_summary, aes(day, mean_y, color = type_chr))+
  
  geom_line(data = sub_summary, aes(day, mean_y, color = type_chr, linetype = as.factor(subnum)), alpha = .3)+
  
  guides(linetype_chr = "none")+
  labs(x = "Day", y = "Mean Log RT", color="", shape = "")+
  theme(panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank(),
        panel.grid.minor.y = element_blank(),
        legend.position = "bottom",
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical",
        axis.text = element_text(size=14),
        legend.text = element_text(size=14),
        axis.title = element_text(size = 16))+
  scale_x_continuous(breaks=seq(1,11,1))+
  scale_color_brewer(palette = "Dark2", labels = c("Hyper-trained", "Regular Exposure")) +
  scale_linetype_manual(values = rep("solid", 6))+
  guides(linetype = "none")+
  annotate("rect", xmin = 2.75, xmax = 3.25, ymin = -.7, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 6.75, xmax = 7.25, ymin = -.7, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 10.75, xmax = 11.25, ymin = -.7, ymax = 1,
           alpha = .25,fill = "gray")

# ggsave(file=paste0(fig_out_path, 'yn_groupAndSub_meanLogRTOverDays.jpg'), p, height = 5, width=6, units="in")
# p

In [ ]:
fn_draws = paste0(inputs_path, 'yn_rtOverDays_draws.csv')

if(file.exists(fn_draws)){
  yn_rt_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_yn_clean$subnum)
  normMax = 1
  normMin = 0
  
  yn_rt_draws = tibble()
  
  for(cur_sub in subnums){
  
    cur_dat = data_yn_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type = as.factor(type),
             norm_day = (normMax - normMin) / (max(day) - min(day)) * (day - max(day)) + (normMax),
             log_rt = log(rt))
    
    cur_m <- brm(formula = log_rt ~  norm_day * type,
                        data= cur_dat,
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    yn_rt_draws = rbind(yn_rt_draws, cur_draws)
  }
  names(yn_rt_draws) = gsub(":", ".", names(yn_rt_draws))
  write.csv(yn_rt_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}


In [ ]:
fn_draws = paste0(inputs_path, 'bc_rtOverDays_draws.csv')

if(file.exists(fn_draws)){
  bc_rt_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_bc_clean$subnum)
  normMax = 1
  normMin = 0
  
  bc_rt_draws = tibble()
  
  for(cur_sub in subnums){
  
    cur_dat = data_bc_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type = as.factor(typeLeft),
             norm_day = (normMax - normMin) / (max(day) - min(day)) * (day - max(day)) + (normMax),
             log_rt = log(rt))
    
    cur_m <- brm(formula = log_rt ~  norm_day * type,
                        data= cur_dat,
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    bc_rt_draws = rbind(bc_rt_draws, cur_draws)
  }
  names(bc_rt_draws) = gsub(":", ".", names(bc_rt_draws))
  write.csv(bc_rt_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}


In [ ]:
yn_par_summary = as_tibble(yn_rt_draws) %>%
  select(b_Intercept, b_norm_day, b_type1, b_norm_day.type1, subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_day" = "Day [0, 1]",
                    "b_type1" = "Type (RE = 0, HT = 1)",
                    "b_norm_day.type1" = "Day x Type"),
         key = factor(key, levels = c("Intercept", "Day [0, 1]", "Type (RE = 0, HT = 1)", "Day x Type"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "Y/N") 

bc_par_summary = as_tibble(bc_rt_draws) %>%
  select(b_Intercept, b_norm_day, b_type1, b_norm_day.type1, subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_day" = "Day [0, 1]",
                    "b_type1" = "Type (RE = 0, HT = 1)",
                    "b_norm_day.type1" = "Day x Type"),
         key = factor(key, levels = c("Intercept", "Day [0, 1]", "Type (RE = 0, HT = 1)", "Day x Type"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "BC") 

# plim = max(abs(bc_par_summary$l95), abs(yn_par_summary$l95) ,bc_par_summary$h95, yn_par_summary$h95)

p = yn_par_summary %>%
  rbind(bc_par_summary) %>%
  ggplot(aes(m, subnum, color = task, alpha = task)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_grid(.~key, scales = 'free_x') +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - log(RT) ~ Day x Type") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  # scale_x_continuous(limits = c(-plim, plim))+
  scale_x_symmetric(mid= 0)+
  scale_color_manual(values = c("red", "blue"))+
  scale_alpha_manual(values = c(1, 1))+
  guides(alpha = "none")

# ggsave(file=paste0(fig_out_path, 'ynbc_rt_ParEsts.jpg'), p, height = 3, width = 9, units="in")
p

## Modeling tasks together

In [ ]:
p = ggplot()+
  geom_point(data = group_summary, aes(day, mean_y, color = task))+
  
  geom_line(data = group_summary, aes(day, mean_y, color = task))+
  
  geom_line(data = sub_summary, aes(day, mean_y, color = task, linetype = as.factor(subnum)), alpha = .3)+

  facet_wrap(~type_chr)+
  labs(x = "Day", y = "Mean Log RT", color="", linetype = "")+
  theme(panel.grid.minor.x = element_blank(),
        panel.grid.major.x = element_blank(),
        legend.position = "bottom",
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_x_continuous(breaks=seq(1,11,1))+
  scale_color_manual(values = c("red", "blue"), limits = c("BC", "Y/N"))+
  scale_linetype_manual(values = rep("solid", 6))+
  guides(linetype = "none")+
  annotate("rect", xmin = 2.75, xmax = 3.25, ymin = -.7, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 6.75, xmax = 7.25, ymin = -.7, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 10.75, xmax = 11.25, ymin = -.7, ymax = 1,
           alpha = .25,fill = "gray")

# ggsave(file=paste0(fig_out_path, 'ynbc_groupAndSub_rtOverDaysPanelByType.jpg'), p, height = 5, width=12, units="in")
p

In [ ]:
p = ggplot(data = sub_summary)+
  
  geom_point(aes(day, mean_y, color = task)) +

  geom_line(aes(day, mean_y, color = task), alpha = .3)+
  
  geom_errorbar(aes(x = day, color = task, ymin = mean_y - sem_y, ymax = mean_y + sem_y), width=0, alpha = .3)+

  facet_grid(type_chr~subnum)+
  labs(x = "Day", y = "Mean Log RT", color="")+
  theme(panel.grid.minor.x = element_blank(),
        panel.grid.major.x = element_blank(),
        legend.position = "bottom",
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_x_continuous(breaks=seq(1,11,1))+
  scale_color_manual(values = c("red", "blue"), limits = c("BC", "Y/N"))+
  annotate("rect", xmin = 2.75, xmax = 3.25, ymin = -.7, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 6.75, xmax = 7.25, ymin = -.7, ymax = 1,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 10.75, xmax = 11.25, ymin = -.7, ymax = 1,
           alpha = .25,fill = "gray")

# ggsave(file=paste0(fig_out_path, 'ynbc_Sub_rtOverDaysPanelByType.jpg'), p, height = 5, width=12, units="in")
p

In [ ]:
fn_draws = paste0(inputs_path, 'ynbc_rtOverDays_draws.csv')

if(file.exists(fn_draws)){
  ynbc_rt_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_bc_clean$subnum)
  normMax = 1
  normMin = 0
  
  ynbc_rt_draws = tibble()
  
  for(cur_sub in subnums){
  
    cur_dat1 = data_yn_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type = as.factor(type),
             norm_day = (normMax - normMin) / (max(day) - min(day)) * (day - max(day)) + (normMax),
             task = "YN",
             log_rt = log(rt)) %>%
      select(log_rt, norm_day, type, task)
    
    cur_dat2 = data_bc_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type = as.factor(typeLeft),
             norm_day = (normMax - normMin) / (max(day) - min(day)) * (day - max(day)) + (normMax),
             task = "BC",
             log_rt = log(rt)) %>%
      select(log_rt, norm_day, type, task)
    
    cur_dat = rbind(cur_dat1, cur_dat2) %>%
      mutate(task = factor(task, levels = c("YN", "BC")))
    
    cur_m <- brm(formula = log_rt ~  norm_day * type * task,
                        data= cur_dat,
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    ynbc_rt_draws = rbind(ynbc_rt_draws, cur_draws)
  }
  names(ynbc_rt_draws) = gsub(":", ".", names(ynbc_rt_draws))
  write.csv(ynbc_rt_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}

In [ ]:
ynbc_par_summary = as_tibble(ynbc_rt_draws) %>%
  select(b_Intercept, b_norm_day, b_type1, b_taskBC, b_norm_day.type1, b_norm_day.taskBC, b_type1.taskBC, b_norm_day.type1.taskBC, subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_day" = "Day [0, 1]",
                    "b_type1" = "Type (RE = 0, HT = 1)",
                    "b_taskBC" = "Task (YN = 0, BC = 1)",
                    "b_norm_day.type1" = "Day x Type",
                    "b_norm_day.taskBC" = "Day x Task", 
                    "b_type1.taskBC" = "Type x Task", 
                    "b_norm_day.type1.taskBC" = "Day x Type x Task"),
         key = factor(key, levels = c("Intercept", "Day [0, 1]", "Type (RE = 0, HT = 1)", "Task (YN = 0, BC = 1)", "Day x Type", "Day x Task", "Type x Task", "Day x Type x Task"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum)) 

plim = max(abs(ynbc_par_summary$l95), ynbc_par_summary$h95)

p = ynbc_par_summary %>%
  ggplot(aes(m, subnum)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75), color = "purple")+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75), color ="purple")+
  facet_wrap(~key, scales = 'free_x', nrow = 2) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - log(RT) ~ Day x Type x Task") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_continuous(limits = c(-plim, plim))

# ggsave(file=paste0(fig_out_path, 'ynbc_rt_SingleModelParEsts.jpg'), p, height = 5, width = 9, units="in")
p

## RT difference over days

In [ ]:
sub_summary = data_yn_clean %>%
  mutate(log_rt = log(rt)) %>%
  group_by(day, subnum, type_chr) %>%
  summarise(mean_y = mean(log_rt),
            .groups = 'keep') %>%
  spread(type_chr, mean_y) %>%
  mutate(y_diff = HT - RE,
         task = "Y/N",
         sum_type = "sub") %>%
  rbind(data_bc_clean %>%
          mutate(log_rt = log(rt)) %>%
          group_by(day, subnum, type_chr) %>%
          summarise(mean_y = mean(log_rt),
                    .groups = 'keep') %>%
          spread(type_chr, mean_y) %>%
          mutate(y_diff = HT - RE,
                 task = "BC",
                 sum_type = "sub")) %>%
  mutate(task = factor(task, levels = c("Y/N", "BC")))

group_summary = data_yn_clean %>%
  mutate(log_rt = log(rt)) %>%
  group_by(day, type_chr) %>%
  summarise(mean_y = mean(log_rt),
            .groups = 'keep') %>%
  spread(type_chr, mean_y) %>%
  mutate(y_diff = HT - RE,
         task = "Y/N",
         sum_type = "group", subnum = 600) %>%
  rbind(data_bc_clean %>%
          mutate(log_rt = log(rt)) %>%
          group_by(day, type_chr) %>%
          summarise(mean_y = mean(log_rt),
                    .groups = 'keep') %>%
          spread(type_chr, mean_y) %>%
          mutate(y_diff = HT - RE,
                 task = "BC",
                 sum_type = "group", subnum = 600)) %>%
  mutate(task = factor(task, levels = c("Y/N", "BC")))

In [ ]:
p = ggplot()+
  geom_point(data = group_summary, aes(day, y_diff), color = "purple")+
  
  geom_line(data = group_summary, aes(day, y_diff), color = "purple")+
  
  geom_line(data = sub_summary, aes(day, y_diff, linetype = as.factor(subnum)), alpha = .3, color = "purple")+

  facet_wrap(~task)+
  labs(x = "Day", y = "log (RT) Difference (HT - RE)", linetype = "")+
  theme(panel.grid.minor.x = element_blank(),
        panel.grid.major.x = element_blank(),
        legend.position = "bottom",
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_x_continuous(breaks=seq(1,11,1))+
  scale_y_continuous(breaks=seq(-0.5,.5,0.5), limits= c(-0.5, .5))+
  scale_linetype_manual(values = rep("solid", 6))+
  guides(linetype = "none")+
  annotate("rect", xmin = 2.75, xmax = 3.25, ymin = -0.5, ymax = .5,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 6.75, xmax = 7.25, ymin = -0.5, ymax = .5,
           alpha = .25,fill = "gray")+
  annotate("rect", xmin = 10.75, xmax = 11.25, ymin = -0.5, ymax = .5,
           alpha = .25,fill = "gray")

# ggsave(file=paste0(fig_out_path, 'ynbc_groupAndSub_logRTDiffOverDays.jpg'), p, height = 5, width=12, units="in")
p

In [ ]:
data_yn_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      group_by(type_chr, day) %>%
      summarise(mean_y = mean(log(rt)), .groups = "keep") %>%
      spread(type_chr, mean_y) %>%
      ungroup() %>%  
      mutate(y_diff = HT - RE,
             norm_day = (normMax - normMin) / (max(day) - min(day)) * (day - max(day)) + (normMax),
             task = "YN") %>%
      select(y_diff, norm_day, task)

In [ ]:
fn_draws = paste0(inputs_path, 'ynbc_rt_diff_draws.csv')

if(file.exists(fn_draws)){
  ynbc_rt_diff_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_bc_clean$subnum)
  normMax = 1
  normMin = 0
  
  ynbc_rt_diff_draws = tibble()
  
  for(cur_sub in subnums){
  
    cur_dat1 = data_yn_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      group_by(type_chr, day) %>%
      summarise(mean_y = mean(log(rt)), .groups = "keep") %>%
      spread(type_chr, mean_y) %>%
      ungroup() %>%  
      mutate(y_diff = HT - RE,
             norm_day = (normMax - normMin) / (max(day) - min(day)) * (day - max(day)) + (normMax),
             task = "YN") %>%
      select(y_diff, norm_day, task)
    
    cur_dat2 = data_bc_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      group_by(type_chr, day) %>%
      summarise(mean_y = mean(log(rt)), .groups = "keep") %>%
      spread(type_chr, mean_y) %>%
      ungroup() %>%  
      mutate(y_diff = HT - RE,
             norm_day = (normMax - normMin) / (max(day) - min(day)) * (day - max(day)) + (normMax),
             task = "BC") %>%
      select(y_diff, norm_day, task)
    
    cur_dat = rbind(cur_dat1, cur_dat2) %>%
      mutate(task = factor(task, levels = c("YN", "BC")))
    
    cur_m <- brm(formula = y_diff ~  norm_day * task,
                        data= cur_dat,
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    ynbc_rt_diff_draws = rbind(ynbc_rt_diff_draws, cur_draws)
  }
  names(ynbc_rt_diff_draws) = gsub(":", ".", names(ynbc_rt_diff_draws))
  write.csv(ynbc_rt_diff_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub, cur_dat1, cur_dat2)
}

In [ ]:
names(as_tibble(ynbc_rt_diff_draws))[grepl("b_", names(as_tibble(ynbc_rt_diff_draws)))]

In [ ]:
ynbc_par_summary = as_tibble(ynbc_rt_diff_draws) %>%
  select(b_Intercept, b_norm_day, b_taskBC, b_norm_day.taskBC, subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_day" = "Day [0, 1]",
                    "b_taskBC" = "Task (YN = 0, BC = 1)",
                    "b_norm_day.taskBC" = "Day x Task"),
         key = factor(key, levels = c("Intercept", "Day [0, 1]", "Task (YN = 0, BC = 1)", "Day x Task"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum)) 

plim = max(abs(ynbc_par_summary$l95), ynbc_par_summary$h95)

p = ynbc_par_summary %>%
  ggplot(aes(m, subnum)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75), color = "purple")+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75), color ="purple")+
  facet_grid(.~key, scales = 'free_x') +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - RT Difference ~ Day x Task") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_continuous(limits = c(-plim, plim))

# ggsave(file=paste0(fig_out_path, 'ynbc_rt_diff_SingleModelParEsts.jpg'), p, height = 3, width = 9, units="in")
p

# RT over value difference

## Plots

In [ ]:
data_yn_clean = data_yn_clean %>%
  mutate(abs_val_diff_bin_str =  paste0(abs(val_diff_bin)*50-25,":",abs(val_diff_bin)*50+25),
         abs_val_diff_bin_str = ifelse(abs_val_diff_bin_str ==  "-25:25", "0:25", abs_val_diff_bin_str),
         abs_val_diff_bin_str = factor(abs_val_diff_bin_str, levels = c("0:25", "25:75", "75:125", "125:175", "175:225", "225:275")))

In [ ]:
sub_summary = data_yn_clean %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(subnum, time_bin, abs_val_diff_bin_str, type_chr) %>%
  summarise(mean_y = mean(log(rt)),
            sem_y = sd(log(rt))/sqrt(n()),
            .groups="keep") %>%
  mutate(type_chr = ifelse(type_chr == "HT", "Hyper-trained", "Regular Exposure"))

group_summary = data_yn_clean %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(time_bin, abs_val_diff_bin_str, type_chr) %>%
  summarise(mean_y = mean(log(rt)),
            .groups="keep") %>%
  mutate(type_chr = ifelse(type_chr == "HT", "Hyper-trained", "Regular Exposure"))

In [ ]:
p = ggplot() +
  geom_point(data = group_summary, aes(abs_val_diff_bin_str, mean_y, color = type_chr, alpha = time_bin, size = time_bin))+
  geom_line(data = group_summary, aes(abs_val_diff_bin_str, mean_y, color = type_chr, group = time_bin, alpha = time_bin)) +
  facet_wrap(~type_chr)+
  # labs(x = "|Value Stim - Value Reference|", y = "Mean Log RT", title="Y/N Task", alpha="", size = "")+
  labs(x = "|Value Stim - Value Reference|", y = "Mean Log RT", alpha="", size = "")+
  theme(panel.grid.major.x  = element_blank(),
        panel.grid.minor.x  = element_blank(),
        panel.grid.minor.y  = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5), size = 14),
        axis.text.y = element_text(size = 14),
        strip.text = element_text(size = 14),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical",
        legend.text = element_text(size=14),
        axis.title = element_text(size = 16))+
  scale_color_brewer(palette = "Dark2")+
  scale_y_continuous(breaks = seq(-1,1,.5), limits = c(-1, 1))+
  guides(color = "none")+
  scale_alpha_manual(values = c(1, .6, .5, .3, .2))+
  scale_size_manual(values = c(1.5, 2 , 2.5, 3, 3.5))

# ggsave(file=paste0(fig_out_path, 'yn_group_MeanLogRTOverValDiffAcrossTimeSDNSymp.jpg'), p, height = 4, width = 9.5, units="in")

p

In [ ]:
p = ggplot() +
   geom_point(data = sub_summary, aes(abs_val_diff_bin_str, mean_y, color = type_chr, alpha = time_bin, size = time_bin))+
  geom_line(data = sub_summary, aes(abs_val_diff_bin_str, mean_y, color = type_chr, group = time_bin, alpha = time_bin)) +
  geom_errorbar(data = sub_summary, aes(x = abs_val_diff_bin_str, ymin = mean_y-sem_y, ymax = mean_y + sem_y, color = type_chr), width = 0, alpha = .6)+
  facet_grid(type_chr~subnum)+
  labs(x = "|Value Stim - Value Reference|", y = "Mean Log RT", title="Y/N Task", alpha="", size = "")+
  theme(panel.grid.major.x  = element_blank(),
        panel.grid.minor.x  = element_blank(),
        panel.grid.minor.y  = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_brewer(palette = "Dark2")+
  scale_y_continuous(breaks = seq(-1,1,.5), limits = c(-1.3, 1.3))+
  guides(linetype_chr = "none", color = "none")+
  scale_alpha_manual(values = c(1, .6, .5, .3, .2))+
  scale_size_manual(values = c(1.5, 2 , 2.5, 3, 3.5))

# ggsave(file=paste0(fig_out_path, 'yn_subj_MeanLogRTValDiffAcrossTime.jpg'), p, height = 5, width = 12, units="in")
p

In [ ]:
data_bc_clean = data_bc_clean %>%
  mutate(abs_val_diff_bin_str =  paste0(abs(val_diff_bin)*50-25,":",abs(val_diff_bin)*50+25),
         abs_val_diff_bin_str = ifelse(abs_val_diff_bin_str ==  "-25:25", "0:25", abs_val_diff_bin_str),
         abs_val_diff_bin_str = factor(abs_val_diff_bin_str, levels = c("0:25", "25:75", "75:125", "125:175", "175:225", "225:275")))

In [ ]:
sub_summary = data_bc_clean %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(subnum, time_bin, abs_val_diff_bin_str, type_chr) %>%
  summarise(mean_y = mean(log(rt)),
            sem_y = sd(log(rt))/sqrt(n()),
            .groups="keep")

group_summary = data_bc_clean %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(time_bin, abs_val_diff_bin_str, type_chr) %>%
  summarise(mean_y = mean(log(rt)),
            .groups="keep")

In [ ]:
p = ggplot() +
  geom_point(data = group_summary, aes(abs_val_diff_bin_str, mean_y, color = type_chr, alpha = time_bin, size = time_bin))+
  geom_line(data = group_summary, aes(abs_val_diff_bin_str, mean_y, color = type_chr, group = time_bin, alpha = time_bin)) +
  facet_wrap(~type_chr)+
  labs(x = "|Value Stim - Value Reference|", y = "Mean Log RT", title="BC Task", alpha="", size = "")+
  theme(panel.grid.major.x  = element_blank(),
        panel.grid.minor.x  = element_blank(),
        panel.grid.minor.y  = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5), size = 12),
        axis.text.y = element_text(size = 12),
        strip.text = element_text(size = 12),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_brewer(palette = "Dark2")+
  scale_y_continuous(breaks = seq(-1,1,.5), limits = c(-1, 1))+
  guides(color = "none")+
  scale_alpha_manual(values = c(1, .6, .5, .3, .2))+
  scale_size_manual(values = c(1.5, 2 , 2.5, 3, 3.5))

# ggsave(file=paste0(fig_out_path, 'bc_group_MeanLogRTOverValDiffAcrossTime.jpg'), p, height = 4, width = 8, units="in")

p

In [ ]:
p = ggplot() +
  geom_point(data = sub_summary, aes(abs_val_diff_bin_str, mean_y, color = type_chr, alpha = time_bin, size = time_bin))+
  geom_line(data = sub_summary, aes(abs_val_diff_bin_str, mean_y, color = type_chr, group = time_bin, alpha = time_bin)) +
  geom_errorbar(data = sub_summary, aes(x = abs_val_diff_bin_str, ymin = mean_y-sem_y, ymax = mean_y + sem_y, color = type_chr), width = 0, alpha = .6) +
  facet_grid(type_chr~subnum)+
  labs(x = "|Value Stim - Value Reference|", y = "Mean Log RT", title="BC Task", alpha="", size = "")+
  theme(panel.grid.major.x  = element_blank(),
        panel.grid.minor.x  = element_blank(),
        panel.grid.minor.y  = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_brewer(palette = "Dark2")+
  scale_y_continuous(breaks = seq(-1,1,.5), limits = c(-1.35, 1.35))+
  guides(color = "none")+
  scale_alpha_manual(values = c(1, .6, .5, .3, .2))+
  scale_size_manual(values = c(1.5, 2 , 2.5, 3, 3.5))

# ggsave(file=paste0(fig_out_path, 'bc_subj_MeanLogRTValDiffAcrossTime.jpg'), p, height = 5, width = 12, units="in")
p

## Models

In [ ]:
fn_draws = paste0(inputs_path, 'yn_rt_norm_draws.csv')

if(file.exists(fn_draws)){
  yn_rt_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_yn_clean$subnum)
  normMax = 1
  normMin = 0
  
  yn_rt_draws = tibble()
  
  for(cur_sub in subnums){
  
    cur_dat = data_yn_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type = as.factor(type), # Coding of type: doesn't matter if it's a factor vs 0/1 integer
             norm_day = (normMax - normMin) / (max(day) - min(day)) * (day - max(day)) + (normMax),
             log_rt = log(rt)) %>%
      group_by(day) %>% # Grouping by day to normalize valdiff matters. This makes sure every day ranges [-1,1]
      mutate(norm_absValDiff = (normMax - normMin) / (max(abs_val_diff) - min(abs_val_diff)) * (abs_val_diff - max(abs_val_diff)) + (normMax)) %>%
      ungroup() %>%
      select(log_rt, norm_absValDiff, norm_day, type)
    
    cur_m <- brm(formula = log_rt ~ norm_absValDiff * norm_day * type,
                        data= cur_dat,
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    yn_rt_draws = rbind(yn_rt_draws, cur_draws)
  }
  names(yn_rt_draws) = gsub(":", ".", names(yn_rt_draws))
  write.csv(yn_rt_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}


In [ ]:
fn_draws = paste0(inputs_path, 'bc_rt_norm_draws.csv')

if(file.exists(fn_draws)){
  bc_rt_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_bc_clean$subnum)
  normMax = 1
  normMin = 0
  
  bc_rt_draws = tibble()
  
  for(cur_sub in subnums){
  
    cur_dat = data_bc_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type = as.factor(typeLeft), # Coding of type: doesn't matter if it's a factor vs 0/1 integer
             norm_day = (normMax - normMin) / (max(day) - min(day)) * (day - max(day)) + (normMax),
             log_rt = log(rt)) %>%
      group_by(day) %>% # Grouping by day to normalize valdiff matters. This makes sure every day ranges [-1,1]
      mutate(norm_absValDiff = (normMax - normMin) / (max(abs_val_diff) - min(abs_val_diff)) * (abs_val_diff - max(abs_val_diff)) + (normMax)) %>%
      ungroup() %>%
      select(log_rt, norm_absValDiff, norm_day, type)
    
    cur_m <- brm(formula = log_rt ~ norm_absValDiff * norm_day * type,
                        data= cur_dat,
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    bc_rt_draws = rbind(bc_rt_draws, cur_draws)
  }
  names(bc_rt_draws) = gsub(":", ".", names(bc_rt_draws))
  write.csv(bc_rt_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}


In [ ]:
names(as_tibble(yn_rt_draws))[grepl("b_", names(as_tibble(yn_rt_draws)))]

In [ ]:
names(as_tibble(bc_rt_draws))[grepl("b_", names(as_tibble(bc_rt_draws)))]

In [ ]:
yn_par_summary = as_tibble(yn_rt_draws) %>%
  select(b_Intercept, b_norm_absValDiff, b_norm_day, b_type1, b_norm_absValDiff.norm_day, b_norm_absValDiff.type1, b_norm_day.type1, b_norm_absValDiff.norm_day.type1, subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_absValDiff"  = "|Value Diff| [0, 1]",
                    "b_norm_day" = "Day [0, 1]",
                    "b_type1" = "Type (RE = 0, HT = 1)",
                    "b_norm_absValDiff.norm_day" = "|Value Diff| x Day",
                    "b_norm_absValDiff.type1" = "|Value Diff| x Type",
                    "b_norm_day.type1" = "Day x Type",
                    "b_norm_absValDiff.norm_day.type1" = "|Value Diff| x Day x Type"),
         key = factor(key, levels = c("Intercept", "|Value Diff| [0, 1]", "Day [0, 1]", "Type (RE = 0, HT = 1)", "|Value Diff| x Day", "|Value Diff| x Type", "Day x Type", "|Value Diff| x Day x Type"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "Y/N") 

bc_par_summary = as_tibble(bc_rt_draws) %>%
  select(b_Intercept, b_norm_absValDiff, b_norm_day, b_type1, b_norm_absValDiff.norm_day, b_norm_absValDiff.type1, b_norm_day.type1, b_norm_absValDiff.norm_day.type1, subnum) %>%
  gather(key, value, -subnum) %>%
    mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_absValDiff"  = "|Value Diff| [0, 1]",
                    "b_norm_day" = "Day [0, 1]",
                    "b_type1" = "Type (RE = 0, HT = 1)",
                    "b_norm_absValDiff.norm_day" = "|Value Diff| x Day",
                    "b_norm_absValDiff.type1" = "|Value Diff| x Type",
                    "b_norm_day.type1" = "Day x Type",
                    "b_norm_absValDiff.norm_day.type1" = "|Value Diff| x Day x Type"),
         key = factor(key, levels = c("Intercept", "|Value Diff| [0, 1]", "Day [0, 1]", "Type (RE = 0, HT = 1)", "|Value Diff| x Day", "|Value Diff| x Type", "Day x Type", "|Value Diff| x Day x Type"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "BC") 


In [ ]:
# library(lemon)
plim = max(abs(bc_par_summary$l95), abs(yn_par_summary$l95) ,bc_par_summary$h95, yn_par_summary$h95)

p = yn_par_summary %>%
  rbind(bc_par_summary) %>%
  ggplot(aes(m, subnum, color = task, alpha = task)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_wrap(~key, scales = 'free_x', nrow = 2) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - log(RT) ~ |Value Diff| x Day x Type") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_continuous(limits = c(-plim, plim))+
  # scale_x_symmetric(mid=0)+
  scale_color_manual(values = c("red", "blue"))+
  scale_alpha_manual(values = c(1, 1))+
  guides(alpha = "none")

# ggsave(file=paste0(fig_out_path, 'ynbc_normRT_ParEsts.jpg'), p, height = 5, width = 9, units="in")
p

# Continuous increase of value difference effect

## Pooled model comparing value difference slope across time bins

In [ ]:

# days = list("day 1" = c(1), "day 2" = c(2), "day 3" = c(3), "week 2" = c(4, 5, 6, 7, 8), "week 3" = c(9, 10, 11))
data_yn_clean = data_yn_clean %>%
  mutate(time_bin = recode(day, 
                    `1` = "day 1", 
                    `2` = "day 2",
                    `3` = "day 3",
                    `4` = "week 2",
                    `5` = "week 2",
                    `6` = "week 2",
                    `7` = "week 2",
                    `8` = "week 2",
                    `9` = "week 3",
                    `10` = "week 3",
                    `11` = "week 3"),
         time_bin = factor(time_bin))

data_bc_clean = data_bc_clean %>%
  mutate(time_bin = recode(day, 
                    `1` = "day 1", 
                    `2` = "day 2",
                    `3` = "day 3",
                    `4` = "week 2",
                    `5` = "week 2",
                    `6` = "week 2",
                    `7` = "week 2",
                    `8` = "week 2",
                    `9` = "week 3",
                    `10` = "week 3",
                    `11` = "week 3"),
         time_bin = factor(time_bin))

contrasts(data_yn_clean$time_bin)

### Current to all later

Helmert coding: Compare day 1 to later, day 2 to later etc. 

In [ ]:
k = length(unique(data_yn_clean$time_bin)) # number of factor levels

my_helmert = matrix(c(4/5, -1/5, -1/5, -1/5, -1/5,
                      0, 3/4, -1/4, -1/4, -1/4,
                      0, 0, 2/3, -1/3, -1/3,
                      0, 0, 0, 1/2, -1/2), ncol = k-1)
  
my_helmert

In [ ]:
fn_draws = paste0(inputs_path, 'yn_rt_norm_helmert_draws.csv')

if(file.exists(fn_draws)){
  yn_rt_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_yn_clean$subnum)

  normMax = 1
  normMin = 0
  
  yn_rt_draws = tibble()

  for(cur_sub in subnums){
    
    cur_dat = data_yn_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type = as.factor(type), # Coding of type: doesn't matter if it's a factor vs 0/1 integer
             norm_day = (normMax - normMin) / (max(day) - min(day)) * (day - max(day)) + (normMax),
             log_rt = log(rt)) %>%
      group_by(day) %>% # Grouping by day to normalize valdiff matters. This makes sure every day ranges [-1,1]
      mutate(norm_absValDiff = (normMax - normMin) / (max(abs_val_diff) - min(abs_val_diff)) * (abs_val_diff - max(abs_val_diff)) + (normMax)) %>%
      ungroup() %>%
      select(log_rt, norm_absValDiff, time_bin, type)
    
    contrasts(cur_dat$time_bin) = my_helmert
    
    cur_m <- brm(formula = log_rt ~ norm_absValDiff * type * time_bin,
                        data= cur_dat,
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)

    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub

    yn_rt_draws = rbind(yn_rt_draws, cur_draws)

  }
  names(yn_rt_draws) = gsub(":", ".", names(yn_rt_draws))
  write.csv(yn_rt_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}


In [ ]:
fn_draws = paste0(inputs_path, 'bc_rt_norm_helmert_draws.csv')

if(file.exists(fn_draws)){
  bc_rt_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_bc_clean$subnum)
  normMax = 1
  normMin = 0

  bc_rt_draws = tibble()
  
  for(cur_sub in subnums){
  
    cur_dat = data_bc_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type = as.factor(typeLeft), # Coding of type: doesn't matter if it's a factor vs 0/1 integer
             norm_day = (normMax - normMin) / (max(day) - min(day)) * (day - max(day)) + (normMax),
             log_rt = log(rt)) %>%
      group_by(day) %>% # Grouping by day to normalize valdiff matters. This makes sure every day ranges [-1,1]
      mutate(norm_absValDiff = (normMax - normMin) / (max(abs_val_diff) - min(abs_val_diff)) * (abs_val_diff - max(abs_val_diff)) + (normMax)) %>%
      ungroup() %>%
      select(log_rt, norm_absValDiff, time_bin, type)
    
    contrasts(cur_dat$time_bin) = my_helmert
    
    cur_m <- brm(formula = log_rt ~ norm_absValDiff * type * time_bin,
                        data= cur_dat,
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    bc_rt_draws = rbind(bc_rt_draws, cur_draws)
  }
  names(bc_rt_draws) = gsub(":", ".", names(bc_rt_draws))
  write.csv(bc_rt_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}


In [ ]:
names(as_tibble(yn_rt_draws))[grepl("b_", names(as_tibble(yn_rt_draws)))]

In [ ]:
names(as_tibble(bc_rt_draws))[grepl("b_", names(as_tibble(bc_rt_draws)))]

In [ ]:
names(as_tibble(yn_rt_draws) %>% select(-lprior, -lp__, -.chain, -.iteration, -.draw, -sigma))

In [ ]:
names(as_tibble(bc_rt_draws) %>% select(-lprior, -lp__, -.chain, -.iteration, -.draw, -sigma))

In [ ]:
yn_par_summary = as_tibble(yn_rt_draws) %>%
  select(-lprior, -lp__, -.chain, -.iteration, -.draw, -sigma) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_absValDiff"  = "|Value Diff| [0, 1]",
                    "b_type1" = "Type (RE = 0, HT = 1)",
                    "b_time_bin1" = "Day 1 vs Later",
                    "b_time_bin2" = "Day 2 vs Later",
                    "b_time_bin3" = "Day 3 vs Later",
                    "b_time_bin4" = "Week 2 vs Later",
                    "b_norm_absValDiff.type1" = "|Value Diff| x Type",
                    "b_norm_absValDiff.time_bin1" = "|Value Diff| x Day 1 vs Later",
                    "b_norm_absValDiff.time_bin2" = "|Value Diff| x Day 2 vs Later",
                    "b_norm_absValDiff.time_bin3" = "|Value Diff| x Day 3 vs Later",
                    "b_norm_absValDiff.time_bin4" = "|Value Diff| x Week 2 vs Later",
                    "b_type1.time_bin1" = "Type x Day 1 vs Later",
                    "b_type1.time_bin2" = "Type x Day 2 vs Later",
                    "b_type1.time_bin3" = "Type x Day 3 vs Later",
                    "b_type1.time_bin4" = "Type x Week 2 vs Later",
                    "b_norm_absValDiff.type1.time_bin1" = "|Value Diff| x Type x Day 1 vs Later",
                    "b_norm_absValDiff.type1.time_bin2" = "|Value Diff| x Type x Day 2 vs Later",
                    "b_norm_absValDiff.type1.time_bin3" = "|Value Diff| x Type x Day 3 vs Later",
                    "b_norm_absValDiff.type1.time_bin4" = "|Value Diff| x Type x Week 2 vs Later",
                    ),
         key = factor(key, levels = c("Intercept", 
                                      "|Value Diff| [0, 1]", 
                                      "Type (RE = 0, HT = 1)", 
                                      "Day 1 vs Later", 
                                      "Day 2 vs Later", 
                                      "Day 3 vs Later", 
                                      "Week 2 vs Later", 
                                      "|Value Diff| x Type", 
                                      "|Value Diff| x Day 1 vs Later", 
                                      "|Value Diff| x Day 2 vs Later", 
                                      "|Value Diff| x Day 3 vs Later", 
                                      "|Value Diff| x Week 2 vs Later", 
                                      "Type x Day 1 vs Later", 
                                      "Type x Day 2 vs Later", 
                                      "Type x Day 3 vs Later", 
                                      "Type x Week 2 vs Later", 
                                      "|Value Diff| x Type x Day 1 vs Later", 
                                      "|Value Diff| x Type x Day 2 vs Later", 
                                      "|Value Diff| x Type x Day 3 vs Later", 
                                      "|Value Diff| x Type x Week 2 vs Later"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "Y/N") 

bc_par_summary = as_tibble(bc_rt_draws) %>%
  select(-lprior, -lp__, -.chain, -.iteration, -.draw, -sigma) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_absValDiff"  = "|Value Diff| [0, 1]",
                    "b_type1" = "Type (RE = 0, HT = 1)",
                    "b_time_bin1" = "Day 1 vs Later",
                    "b_time_bin2" = "Day 2 vs Later",
                    "b_time_bin3" = "Day 3 vs Later",
                    "b_time_bin4" = "Week 2 vs Later",
                    "b_norm_absValDiff.type1" = "|Value Diff| x Type",
                    "b_norm_absValDiff.time_bin1" = "|Value Diff| x Day 1 vs Later",
                    "b_norm_absValDiff.time_bin2" = "|Value Diff| x Day 2 vs Later",
                    "b_norm_absValDiff.time_bin3" = "|Value Diff| x Day 3 vs Later",
                    "b_norm_absValDiff.time_bin4" = "|Value Diff| x Week 2 vs Later",
                    "b_type1.time_bin1" = "Type x Day 1 vs Later",
                    "b_type1.time_bin2" = "Type x Day 2 vs Later",
                    "b_type1.time_bin3" = "Type x Day 3 vs Later",
                    "b_type1.time_bin4" = "Type x Week 2 vs Later",
                    "b_norm_absValDiff.type1.time_bin1" = "|Value Diff| x Type x Day 1 vs Later",
                    "b_norm_absValDiff.type1.time_bin2" = "|Value Diff| x Type x Day 2 vs Later",
                    "b_norm_absValDiff.type1.time_bin3" = "|Value Diff| x Type x Day 3 vs Later",
                    "b_norm_absValDiff.type1.time_bin4" = "|Value Diff| x Type x Week 2 vs Later",
                    ),
         key = factor(key, levels = c("Intercept", "|Value Diff| [0, 1]", "Type (RE = 0, HT = 1)", "Day 1 vs Later", "Day 2 vs Later", "Day 3 vs Later", "Week 2 vs Later", "|Value Diff| x Type", "|Value Diff| x Day 1 vs Later", "|Value Diff| x Day 2 vs Later", "|Value Diff| x Day 3 vs Later", "|Value Diff| x Week 2 vs Later", "Type x Day 1 vs Later", "Type x Day 2 vs Later", "Type x Day 3 vs Later", "Type x Week 2 vs Later", "|Value Diff| x Type x Day 1 vs Later", "|Value Diff| x Type x Day 2 vs Later", "|Value Diff| x Type x Day 3 vs Later", "|Value Diff| x Type x Week 2 vs Later"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "BC") 


In [ ]:
p = yn_par_summary %>%
  rbind(bc_par_summary) %>%
  ggplot(aes(m, subnum, color = task, alpha = task)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_wrap(~key, scales = 'free_x', nrow = 2, labeller = labeller(key = label_wrap_gen(10))) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - log(RT) ~ |Value Diff| x Time Bin x Type") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_symmetric(mid=0)+
  scale_color_manual(values = c("red", "blue"))+
  scale_alpha_manual(values = c(1, 1))+
  guides(alpha = "none")

# ggsave(file=paste0(fig_out_path, 'ynbc_rt_helmert_ParEsts.jpg'), p, height = 6, width = 12, units="in")
p

### Successive levels

Forward difference coding: Compare day 1 to day 2, day 2 to day 3 etc


In [ ]:
my_fwd_diff = matrix(c(4/5, -1/5, -1/5, -1/5, -1/5,
                       3/5, 3/5, -2/5, -2/5, -2/5,
                       2/5, 2/5, 2/5, -3/5, -3/5,
                       1/5, 1/5, 1/5, 1/5, -4/5), ncol = k-1)

my_fwd_diff

In [ ]:
fn_draws = paste0(inputs_path, 'yn_rt_norm_fwd_diff_draws.csv')

if(file.exists(fn_draws)){
  yn_rt_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_yn_clean$subnum)

  normMax = 1
  normMin = 0
  
  yn_rt_draws = tibble()

  for(cur_sub in subnums){
    
    cur_dat = data_yn_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type = as.factor(type), # Coding of type: doesn't matter if it's a factor vs 0/1 integer
             norm_day = (normMax - normMin) / (max(day) - min(day)) * (day - max(day)) + (normMax),
             log_rt = log(rt)) %>%
      group_by(day) %>% # Grouping by day to normalize valdiff matters. This makes sure every day ranges [-1,1]
      mutate(norm_absValDiff = (normMax - normMin) / (max(abs_val_diff) - min(abs_val_diff)) * (abs_val_diff - max(abs_val_diff)) + (normMax)) %>%
      ungroup() %>%
      select(log_rt, norm_absValDiff, time_bin, type)
    
    contrasts(cur_dat$time_bin) = my_fwd_diff
    
    cur_m <- brm(formula = log_rt ~ norm_absValDiff * type * time_bin,
                        data= cur_dat,
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)

    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub

    yn_rt_draws = rbind(yn_rt_draws, cur_draws)

  }
  names(yn_rt_draws) = gsub(":", ".", names(yn_rt_draws))
  write.csv(yn_rt_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}


In [ ]:
fn_draws = paste0(inputs_path, 'bc_rt_norm_fwd_diff_draws.csv')

if(file.exists(fn_draws)){
  bc_rt_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_bc_clean$subnum)
  normMax = 1
  normMin = 0

  bc_rt_draws = tibble()
  
  for(cur_sub in subnums){
  
    cur_dat = data_bc_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type = as.factor(typeLeft), # Coding of type: doesn't matter if it's a factor vs 0/1 integer
             norm_day = (normMax - normMin) / (max(day) - min(day)) * (day - max(day)) + (normMax),
             log_rt = log(rt)) %>%
      group_by(day) %>% # Grouping by day to normalize valdiff matters. This makes sure every day ranges [-1,1]
      mutate(norm_absValDiff = (normMax - normMin) / (max(abs_val_diff) - min(abs_val_diff)) * (abs_val_diff - max(abs_val_diff)) + (normMax)) %>%
      ungroup() %>%
      select(log_rt, norm_absValDiff, time_bin, type)
    
    contrasts(cur_dat$time_bin) = my_fwd_diff
    
    cur_m <- brm(formula = log_rt ~ norm_absValDiff * type * time_bin,
                        data= cur_dat,
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    bc_rt_draws = rbind(bc_rt_draws, cur_draws)
  }
  names(bc_rt_draws) = gsub(":", ".", names(bc_rt_draws))
  write.csv(bc_rt_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}


In [ ]:
names(as_tibble(yn_rt_draws))[grepl("b_", names(as_tibble(yn_rt_draws)))]

In [ ]:
names(as_tibble(bc_rt_draws))[grepl("b_", names(as_tibble(bc_rt_draws)))]

In [ ]:
names(as_tibble(yn_rt_draws))

In [ ]:
yn_par_summary = as_tibble(yn_rt_draws) %>%
  select(-lprior, -lp__, -.chain, -.iteration, -.draw, -sigma) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_absValDiff"  = "|Value Diff| [0, 1]",
                    "b_type1" = "Type (RE = 0, HT = 1)",
                    "b_time_bin1" = "Day 1 vs Day 2",
                    "b_time_bin2" = "Day 2 vs Day 3",
                    "b_time_bin3" = "Day 3 vs Week 2",
                    "b_time_bin4" = "Week 2 vs Week 3",
                    "b_norm_absValDiff.type1" = "|Value Diff| x Type",
                    "b_norm_absValDiff.time_bin1" = "|Value Diff| x Day 1 vs Day 2",
                    "b_norm_absValDiff.time_bin2" = "|Value Diff| x Day 2 vs Day 3",
                    "b_norm_absValDiff.time_bin3" = "|Value Diff| x Day 3 vs Week 2",
                    "b_norm_absValDiff.time_bin4" = "|Value Diff| x Week 2 vs Week 3",
                    "b_type1.time_bin1" = "Type x Day 1 vs Day 2",
                    "b_type1.time_bin2" = "Type x Day 2 vs Day 3",
                    "b_type1.time_bin3" = "Type x Day 3 vs Week 2",
                    "b_type1.time_bin4" = "Type x Week 2 vs Week 3",
                    "b_norm_absValDiff.type1.time_bin1" = "|Value Diff| x Type x Day 1 vs Day 2",
                    "b_norm_absValDiff.type1.time_bin2" = "|Value Diff| x Type x Day 2 vs Day 3",
                    "b_norm_absValDiff.type1.time_bin3" = "|Value Diff| x Type x Day 3 vs Week 2",
                    "b_norm_absValDiff.type1.time_bin4" = "|Value Diff| x Type x Week 2 vs Week 3",
                    ),
         key = factor(key, levels = c("Intercept", 
                                      "|Value Diff| [0, 1]", 
                                      "Type (RE = 0, HT = 1)", 
                                      "Day 1 vs Day 2", 
                                      "Day 2 vs Day 3", 
                                      "Day 3 vs Week 2", 
                                      "Week 2 vs Week 3", 
                                      "|Value Diff| x Type", 
                                      "|Value Diff| x Day 1 vs Day 2", 
                                      "|Value Diff| x Day 2 vs Day 3", 
                                      "|Value Diff| x Day 3 vs Week 2", 
                                      "|Value Diff| x Week 2 vs Week 3", 
                                      "Type x Day 1 vs Day 2", 
                                      "Type x Day 2 vs Day 3", 
                                      "Type x Day 3 vs Week 2", 
                                      "Type x Week 2 vs Week 3", 
                                      "|Value Diff| x Type x Day 1 vs Day 2", 
                                      "|Value Diff| x Type x Day 2 vs Day 3", 
                                      "|Value Diff| x Type x Day 3 vs Week 2", 
                                      "|Value Diff| x Type x Week 2 vs Week 3"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "Y/N") 

bc_par_summary = as_tibble(bc_rt_draws) %>%
  select(-lprior, -lp__, -.chain, -.iteration, -.draw, -sigma) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_absValDiff"  = "|Value Diff| [0, 1]",
                    "b_type1" = "Type (RE = 0, HT = 1)",
                    "b_time_bin1" = "Day 1 vs Day 2",
                    "b_time_bin2" = "Day 2 vs Day 3",
                    "b_time_bin3" = "Day 3 vs Week 2",
                    "b_time_bin4" = "Week 2 vs Week 3",
                    "b_norm_absValDiff.type1" = "|Value Diff| x Type",
                    "b_norm_absValDiff.time_bin1" = "|Value Diff| x Day 1 vs Day 2",
                    "b_norm_absValDiff.time_bin2" = "|Value Diff| x Day 2 vs Day 3",
                    "b_norm_absValDiff.time_bin3" = "|Value Diff| x Day 3 vs Week 2",
                    "b_norm_absValDiff.time_bin4" = "|Value Diff| x Week 2 vs Week 3",
                    "b_type1.time_bin1" = "Type x Day 1 vs Day 2",
                    "b_type1.time_bin2" = "Type x Day 2 vs Day 3",
                    "b_type1.time_bin3" = "Type x Day 3 vs Week 2",
                    "b_type1.time_bin4" = "Type x Week 2 vs Week 3",
                    "b_norm_absValDiff.type1.time_bin1" = "|Value Diff| x Type x Day 1 vs Day 2",
                    "b_norm_absValDiff.type1.time_bin2" = "|Value Diff| x Type x Day 2 vs Day 3",
                    "b_norm_absValDiff.type1.time_bin3" = "|Value Diff| x Type x Day 3 vs Week 2",
                    "b_norm_absValDiff.type1.time_bin4" = "|Value Diff| x Type x Week 2 vs Week 3",
                    ),
         key = factor(key, levels = c("Intercept", "|Value Diff| [0, 1]", "Type (RE = 0, HT = 1)", "Day 1 vs Day 2", "Day 2 vs Day 3", "Day 3 vs Week 2", "Week 2 vs Week 3", "|Value Diff| x Type", "|Value Diff| x Day 1 vs Day 2", "|Value Diff| x Day 2 vs Day 3", "|Value Diff| x Day 3 vs Week 2", "|Value Diff| x Week 2 vs Week 3", "Type x Day 1 vs Day 2", "Type x Day 2 vs Day 3", "Type x Day 3 vs Week 2", "Type x Week 2 vs Week 3", "|Value Diff| x Type x Day 1 vs Day 2", "|Value Diff| x Type x Day 2 vs Day 3", "|Value Diff| x Type x Day 3 vs Week 2", "|Value Diff| x Type x Week 2 vs Week 3"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "BC") 


In [ ]:
p = yn_par_summary %>%
  rbind(bc_par_summary) %>%
  ggplot(aes(m, subnum, color = task, alpha = task)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_wrap(~key, scales = 'free_x', nrow = 2, labeller = labeller(key = label_wrap_gen(10))) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - log(RT) ~ |Value Diff| x Time Bin x Type") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_symmetric(mid=0)+
  scale_color_manual(values = c("red", "blue"))+
  scale_alpha_manual(values = c(1, 1))+
  guides(alpha = "none")

# ggsave(file=paste0(fig_out_path, 'ynbc_rt_fwd_diff_ParEsts.jpg'), p, height = 6, width = 12, units="in")
p